# Библиотеки

In [1]:
# !pip install dbfread
# !pip install simpledbf
# !pip install xlrd

import pandas as pd
import numpy as np
import os
from dbfread import DBF
from simpledbf import Dbf5
import xlrd
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt
import seaborn as sb
import scipy.stats
import plotly.express as px

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)

# Подготовка 

Так как при открытии DBF-файла с регистрационными номерами банка и их названиями не считывались регистрационные номера размером меньше, чем четыре знака, то пришлось эти файлы вручную объединять в Excel и удалять дубликаты. Загрузим получившийся файл со всеми названиями банков, с 2014 гг.:

In [3]:
reg_bank = pd.read_excel(r"C:\Users\annaz\Documents\Диплом\BANKS_REGN.xlsx", sheet_name='REGN')
reg_bank['NAME_B'] = reg_bank['NAME_B'].str.split('_').str[0]
reg_bank = reg_bank.rename(columns={'NAME_B': 'BANK'})
reg_bank.head()

,DATE,REGN,BANK
0,102-20250201,1,ЮНИКРЕДИТ БАНК
1,102-20250201,1000,ВТБ
2,102-20250201,101,БКС БАНК
3,102-20250201,1049,ХАКАССКИЙ МУНИЦИПАЛЬНЫЙ БАНК
4,102-20250201,1067,РЕАЛИСТ


Загрузим данные о названии строк отчета о прибылях и убытках (форма 102) на 01.01.2025

In [5]:
dbf_file_report = 'C:/Users/annaz/Documents/Диплом/Форма 102/102-20250101/SPRAV1.dbf'
table_report = DBF(dbf_file_report, encoding='cp866') # Укажите кодировку, если необходимо

df_report = pd.DataFrame(iter(table_report))
df_report.head()

,NOM,PRSTR,CODE,NAME,NAME1
0,1,20,,"Часть 1. Процентные доходы, доходы от корректи...",
1,2,20,,Раздел 1. Процентные доходы,
2,3,120,11000,Итого по разделу 1,
3,4,20,,Раздел 2. Комиссионные доходы,
4,5,120,12000,Итого по разделу 2,


Заметим, что напротив значения CODE отражается "Итого по разделу Х", а не название Раздела Х. Оно находится на одну строку выше
Поэтому сделаем сдвиг на одну строку вверх:

In [17]:
PnL = []
for i in range(1, len(df_report)):
    if pd.notna(df_report['CODE'].iloc[i]):
        correct_name = df_report['NAME'].iloc[i-1]
        PnL.append({'CODE': df_report['CODE'].iloc[i], 'correct_name': correct_name})

report_PL = pd.DataFrame(new_data)
report_PL = report_PL[report_PL['CODE'] != '']
report_PL.to_excel('C:/Users/annaz/Documents/Диплом/new_df.xlsx', index=False)

Проверив файл, были внесены некоторые правки, поэтому воспользуемся скорректированным файлом.

In [7]:
report_PnL = pd.read_excel(r'C:\Users\annaz\Documents\Диплом\report_PnL.xlsx')
report_PnL.head()

,CODE,NAME
0,11000,Раздел 1. Процентные доходы
1,12000,Раздел 2. Комиссионные доходы
2,13000,"Раздел 3. Корректировки, увеличивающие процент..."
3,14000,"Раздел 4. Корректировки, уменьшающие процентны..."
4,15000,Раздел 5. Доходы от восстановления (уменьшения...


# ЦИКЛ для PnL (ежеквартально), 102

Нет данных за весь 2022 год и первый квартал 2023 г

In [188]:
way_102 = ("C:/Users/annaz/Documents/Диплом/Форма 102/")

file_paths = []
count = 0
all_dfs = []

for path, dirs, files in os.walk(way_102):
    for file in files:
        file_data = file
        file_bank = file
        if file_data.lower().endswith('_p1.dbf'):
            file_path = os.path.join(path, file_data)
            print(file_data)
            count+=1            

            dbf = DBF(file_path, encoding='cp866')  
            df = pd.DataFrame(iter(dbf))
            df['DT'] = file_path.split('-')[1][:8]
            all_dfs.append(df)

print(count)
df = pd.concat(all_dfs, ignore_index=True)
df = df.rename(columns={'DT':'DATE', 'SIM_ITOGO': 'VALUE'})
df = df[['DATE', 'REGN', 'CODE', 'VALUE']]

df.to_csv(r'C:\Users\annaz\Documents\Диплом\Промежуточные датафреймы\df.csv', index=False)
df

42009_P1.DBF
12010_P1.DBF
22010_P1.DBF
32010_P1.DBF
42010_P1.DBF
12011_P1.DBF
22011_P1.DBF
32011_P1.DBF
42011_P1.DBF
12012_P1.DBF
22012_P1.DBF
32012_P1.DBF
42012_P1.DBF
12013_P1.DBF
22013_P1.DBF
32013_P1.DBF
42013_P1.DBF
12014_P1.DBF
22014_P1.DBF
32014_P1.DBF
42014_P1.DBF
12015_P1.DBF
22015_P1.DBF
32015_P1.DBF
42015_P1.DBF
12016_P1.DBF
22016_P1.DBF
32016_P1.DBF
42016_P1.DBF
12017_P1.DBF
22017_P1.DBF
32017_P1.DBF
42017_P1.DBF
12018_P1.DBF
22018_P1.DBF
32018_P1.DBF
42018_P1.DBF
12019_P1.DBF
22019_P1.DBF
32019_P1.DBF
42019_P1.DBF
12020_P1.DBF
22020_P1.DBF
32020_P1.DBF
42020_P1.DBF
12021_P1.DBF
22021_P1.DBF
32021_P1.DBF
42021_P1.DBF
22023_P1.dbf
32023_P1.dbf
42023_P1.dbf
12024_P1.dbf
22024_P1.dbf
32024_P1.dbf
42024_P1.dbf
56


C:\Users\annaz\AppData\Local\Temp\ipykernel_8044\3711181553.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(all_dfs, ignore_index=True)


,DATE,REGN,CODE,VALUE
0,20100101,1,01000,7057006.0
1,20100101,1,02000,0.0
2,20100101,1,10000,744909471.0
3,20100101,1,10001,67406862.0
4,20100101,1,10002,676818717.0
...,...,...,...,...
27784505,20250101,328,81202,0.0
27784506,20250101,1281,81202,0.0
27784507,20250101,2722,81202,14630.0
27784508,20250101,3294,81202,0.0


In [190]:
df[df.DATE == '20211001']

,DATE,REGN,CODE,VALUE
26230392,20211001,1,01000,18046268.0
26230393,20211001,1,02000,0.0
26230394,20211001,1,03000,1653641.0
26230395,20211001,1,04000,0.0
26230396,20211001,1,10001,87249735.0
...,...,...,...,...
26954249,20211001,3540,72909,0.0
26954250,20211001,3540,81101,4058.0
26954251,20211001,3540,81102,0.0
26954252,20211001,3540,81201,50825.0


Пропусков немного: только в значениях показателей 1302 пропусков

In [192]:
df.isna().sum()

DATE        0
REGN        0
CODE        0
VALUE    1293
dtype: int64

Поменяем тип данных для CODE, чтобы потом было удобно соединять по нему

In [194]:
df['CODE'] = df['CODE'].astype(int)
df.dtypes

DATE      object
REGN       int64
CODE       int32
VALUE    float64
dtype: object

### Панель + отбрасываем колокни, в которых пропусков больше 90%

In [9]:
df = pd.read_csv(r'C:\Users\annaz\Documents\Диплом\Промежуточные датафреймы\df.csv')

In [222]:
pivot_df = df.pivot_table(index=['DATE', 'REGN'], columns='CODE', values='VALUE', aggfunc='sum')
pivot_df = pivot_df.reset_index()
pivot_df.head()

CODE,DATE,REGN,1000,2000,3000,4000,10000,10001,10002,10003,...,72904,72905,72906,72907,72908,72909,81101,81102,81201,81202
0,20100101,1,7057006.0,0.0,NaN,NaN,744909471.0,67406862.0,676818717.0,683892.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20100101,5,35980.0,0.0,NaN,NaN,3577977.0,828430.0,2744820.0,4727.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20100101,18,0.0,518299.0,NaN,NaN,12460604.0,4134606.0,8284129.0,41869.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20100101,21,8574.0,0.0,NaN,NaN,444700.0,151319.0,292503.0,878.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20100101,23,0.0,65139.0,NaN,NaN,5720451.0,2654339.0,1974186.0,1091926.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Оставим только те колонки, в которых пропусков меньше 10% в колонке по всем наблюдениям. То есть удаляем те колонки, в которых пропусков больше 90% от всех наблюдений

In [224]:
data_PnL = pivot_df
missing_ratio = data_PnL.isnull().sum() / data_PnL.shape[0]
threshold = 0.1
rows_to_save = missing_ratio[missing_ratio < threshold]  # кандидаты на сохранение колонок
rows_to_save.index

Index(['DATE', 'REGN',   1000,   2000,  10001,  10002,  10003,  11000,  11100,
        11112,  11114,  11115,  12000,  12100,  12102,  12200,  13000,  14000,
        15000,  15100,  16000,  21000,  21300,  22000,  22100,  23000,  24000,
        24100,  25000,  25200,  25300,  25302,  26000,  26101,  26201,  26301,
        27000,  27300,  28000],
      dtype='object', name='CODE')

## Добавляем название банка

In [226]:
data_PnL = pd.merge(data_PnL, reg_bank[['BANK', 'REGN']], on='REGN', how='left')
data_PnL = data_PnL[['DATE', 'REGN', 'BANK',   1000,   2000,  10001,  10002,  10003,  11000,  12000,
        13000,  14000,  15000,  16000,  21000,  22000,  23000,  24000,  25000, 27000]]
data_PnL.head()

,DATE,REGN,BANK,1000,2000,10001,10002,10003,11000,12000,13000,14000,15000,16000,21000,22000,23000,24000,25000,27000
0,20100101,1,ЮНИКРЕДИТ БАНК,7057006.0,0.0,67406862.0,676818717.0,683892.0,44723863.0,22682999.0,1226587.0,29016.0,550181541.0,125381573.0,29778064.0,21184997.0,1268204.0,547702745.0,127289615.0,1674774.0
1,20100101,5,"Банк ""Кредит-Москва"" (ПАО)",35980.0,0.0,828430.0,2744820.0,4727.0,422557.0,405873.0,153676.0,75.0,2453204.0,137865.0,256249.0,330283.0,31761.0,2434456.0,189583.0,641.0
2,20100101,18,"ОАО КБ ""Стройкредит""",0.0,518299.0,4134606.0,8284129.0,41869.0,1484656.0,2649950.0,60642.0,1622.0,6519983.0,1701882.0,1020191.0,2550531.0,10523.0,6151173.0,2399101.0,2771.0
3,20100101,21,ПРИМОРСКИЙ ТЕРРИТОРИАЛЬНЫЙ,8574.0,0.0,151319.0,292503.0,878.0,97277.0,54042.0,0.0,0.0,49462.0,243041.0,21896.0,10944.0,0.0,50938.0,239624.0,1082.0
4,20100101,23,АО Банк АВБ,0.0,65139.0,2654339.0,1974186.0,1091926.0,2281616.0,372723.0,6230.0,6916.0,1352310.0,608730.0,1579950.0,158720.0,16372.0,1362143.0,923421.0,1082101.0


Подтянем названия колонок по строке CODE и переименуем колонки в более удобные названия

In [230]:
rename_dict = dict(zip(report_PnL['CODE'], report_PnL['NAME']))
data_PnL = data_PnL.rename(columns=rename_dict)
data_PnL = data_PnL.rename(columns={'DATE':'DATE', 'BANK': 'BANK',
       'Прибыль до налогообложения (строка "Всего доходов (части 1-2)" минус строка "Всего расходов (части 3-4)")' : 'Прибыль до налогообложения',
       'Убыток до налогообложения (строка "Всего расходов (части 3-4)" минус строка "Всего доходов (части 1-2)")' : 'Убыток до налогообложения',
       'Всего по Части 1 "Процентные доходы, доходы от корректировок и от восстановления (уменьшения) резервов на возможные потери" (разделы 1-7)' : 'ВСЕГО Процентные доходы',
       'Всего по Части 2 "Операционные доходы" (разделы 1-9)' : 'Операционные доходы',
       'Всего по Части 3 "Процентные расходы, расходы от корректировок и расходы по формированию резервов на возможные потери" (разделы 1-8)' : 'Процентные расходы',
       'Раздел 1. Процентные доходы' : 'Процентные доходы', 'Раздел 2. Комиссионные доходы' : 'Комиссионные доходы',
       'Раздел 3. Корректировки, увеличивающие процентные доходы, на разницу между процентными доходами за отчетный период, рассчитанными в соответствии с применением эффективной процентной ставки, и процентными доходами, начисленными без применения эффективной' : 'Корректировки, увеличивающие процентные доходы',
       'Раздел 4. Корректировки, уменьшающие процентные расходы, на разницу между процентными расходами за отчетный период, рассчитанными в соответствии с применением эффективной процентной ставки, и процентными расходами, начисленными без применения эффективной' : 'Корректировки, уменьшающие процентные расходы',
       'Раздел 5. Доходы от восстановления (уменьшения) резервов на возможные потери': 'Доходы от восстановления (уменьшения) резервов на возможные потери',
       'Раздел 6. Премии, уменьшающие процентные расходы': 'Премии, уменьшающие процентные расходы',
       'Раздел 1. Доходы (кроме процентных) от операций с предоставленными кредитами и прочими размещенными средствами': 'Доходы (кроме процентных) от операций с предоставленными кредитами и прочими размещенными средствами',
       'Раздел 2. Доходы (кроме процентных) от операций с приобретенными долговыми ценными бумагами': 'Доходы (кроме процентных) от операций с приобретенными долговыми ценными бумагами',
       'Раздел 3. Доходы от операций с приобретенными долевыми ценными бумагами и цифровыми финансовыми активами': 'Доходы от операций с приобретенными долевыми ценными бумагами и цифровыми финансовыми активами',
       'Раздел 4. Доходы от операций с полученными кредитами, а также с прочими привлеченными средствами и выпущенными долговыми ценными бумагами': 'Доходы от операций с полученными кредитами, а также с прочими привлеченными средствами и выпущенными долговыми ценными бумагами',
       'Раздел 5. Доходы от операций с производными финансовыми инструментами, от операций хеджирования и от операций с цифровыми финансовыми активами' : 'Доходы от операций с производными финансовыми инструментами, от операций хеджирования и от операций с цифровыми финансовыми активами',
       'Раздел 7. Комиссионные и аналогичные доходы' : 'Комиссионные и аналогичные доходы'})

data_PnL.head()

,DATE,REGN,BANK,Прибыль до налогообложения,Убыток до налогообложения,ВСЕГО Процентные доходы,Операционные доходы,Процентные расходы,Процентные доходы,Комиссионные доходы,"Корректировки, увеличивающие процентные доходы","Корректировки, уменьшающие процентные расходы",Доходы от восстановления (уменьшения) резервов на возможные потери,"Премии, уменьшающие процентные расходы",Доходы (кроме процентных) от операций с предоставленными кредитами и прочими размещенными средствами,Доходы (кроме процентных) от операций с приобретенными долговыми ценными бумагами,Доходы от операций с приобретенными долевыми ценными бумагами и цифровыми финансовыми активами,"Доходы от операций с полученными кредитами, а также с прочими привлеченными средствами и выпущенными долговыми ценными бумагами","Доходы от операций с производными финансовыми инструментами, от операций хеджирования и от операций с цифровыми финансовыми активами",Комиссионные и аналогичные доходы
0,20100101,1,ЮНИКРЕДИТ БАНК,7057006.0,0.0,67406862.0,676818717.0,683892.0,44723863.0,22682999.0,1226587.0,29016.0,550181541.0,125381573.0,29778064.0,21184997.0,1268204.0,547702745.0,127289615.0,1674774.0
1,20100101,5,"Банк ""Кредит-Москва"" (ПАО)",35980.0,0.0,828430.0,2744820.0,4727.0,422557.0,405873.0,153676.0,75.0,2453204.0,137865.0,256249.0,330283.0,31761.0,2434456.0,189583.0,641.0
2,20100101,18,"ОАО КБ ""Стройкредит""",0.0,518299.0,4134606.0,8284129.0,41869.0,1484656.0,2649950.0,60642.0,1622.0,6519983.0,1701882.0,1020191.0,2550531.0,10523.0,6151173.0,2399101.0,2771.0
3,20100101,21,ПРИМОРСКИЙ ТЕРРИТОРИАЛЬНЫЙ,8574.0,0.0,151319.0,292503.0,878.0,97277.0,54042.0,0.0,0.0,49462.0,243041.0,21896.0,10944.0,0.0,50938.0,239624.0,1082.0
4,20100101,23,АО Банк АВБ,0.0,65139.0,2654339.0,1974186.0,1091926.0,2281616.0,372723.0,6230.0,6916.0,1352310.0,608730.0,1579950.0,158720.0,16372.0,1362143.0,923421.0,1082101.0


## Отбираем нужные строки PnL

Добавим необходимые строки:

Номер статьи в отчете - Название статьи в отчете

61101 - Прибыль после налогообложения

31001 -  Прибыль после налогообложения (после 2016)

61102 - Убыток после налогообложения

31002 - Убыток после налогообложения (после 2016)

(3000 - 4000) - Налог на прибыль 

28101- Налог на прибыль (после 2016)

10003 - Процентные расходы

20001 - Процентные расходы (после 2016)

10004 - Операционные расходы

20002 - Операционные расходы(после 2016)

In [232]:
pd.set_option('display.max_columns', None)
check = pivot_df[['DATE','REGN', 61101, 61102, 31001, 31002, 3000, 4000, 28101, 10003, 10004, 20001, 20002]]

data_PnL = pd.merge(data_PnL, check[['DATE','REGN', 61101, 61102, 31001, 31002, 3000, 4000, 28101, 10003, 10004, 20001, 20002]], on=['REGN', 'DATE'], how='left')
data_PnL['Прибыль после налогообложения'] = data_PnL[61101].fillna(data_PnL[31001])
data_PnL['Убыток после налогообложения'] = data_PnL[61102].fillna(data_PnL[31002])
data_PnL['Финансовый результат'] = data_PnL['Прибыль после налогообложения'] - data_PnL['Убыток после налогообложения']
data_PnL['Налог на прибыль'] = data_PnL[3000] - data_PnL[4000]
data_PnL['Налог на прибыль'] = data_PnL['Налог на прибыль'].fillna(data_PnL[28101])

data_PnL['Процентные расходы'] = data_PnL[10003].fillna(data_PnL[20001])
data_PnL['Операционные расходы'] = data_PnL[10004].fillna(data_PnL[20002])

columns_to_drop  = [61101, 61102, 31001, 31002, 3000, 4000, 28101, 10003, 10004, 20001, 20002]
data_PnL = data_PnL.drop(columns_to_drop, axis=1)

# # Переводим данные в млн. руб.
# numeric_columns = data_PnL.select_dtypes(include=['int64', 'float64']).columns 
# numeric_columns = numeric_columns.drop('REGN')
# data_PnL[numeric_columns] = data_PnL[numeric_columns].div(1000)
data_PnL.head()

,DATE,REGN,BANK,Прибыль до налогообложения,Убыток до налогообложения,ВСЕГО Процентные доходы,Операционные доходы,Процентные расходы,Процентные доходы,Комиссионные доходы,"Корректировки, увеличивающие процентные доходы","Корректировки, уменьшающие процентные расходы",Доходы от восстановления (уменьшения) резервов на возможные потери,"Премии, уменьшающие процентные расходы",Доходы (кроме процентных) от операций с предоставленными кредитами и прочими размещенными средствами,Доходы (кроме процентных) от операций с приобретенными долговыми ценными бумагами,Доходы от операций с приобретенными долевыми ценными бумагами и цифровыми финансовыми активами,"Доходы от операций с полученными кредитами, а также с прочими привлеченными средствами и выпущенными долговыми ценными бумагами","Доходы от операций с производными финансовыми инструментами, от операций хеджирования и от операций с цифровыми финансовыми активами",Комиссионные и аналогичные доходы,Прибыль после налогообложения,Убыток после налогообложения,Финансовый результат,Налог на прибыль,Операционные расходы
0,20100101,1,ЮНИКРЕДИТ БАНК,7057006.0,0.0,67406862.0,676818717.0,683892.0,44723863.0,22682999.0,1226587.0,29016.0,550181541.0,125381573.0,29778064.0,21184997.0,1268204.0,547702745.0,127289615.0,1674774.0,5799474.0,0.0,5799474.0,1257532.0,685214630.0
1,20100101,5,"Банк ""Кредит-Москва"" (ПАО)",35980.0,0.0,828430.0,2744820.0,4727.0,422557.0,405873.0,153676.0,75.0,2453204.0,137865.0,256249.0,330283.0,31761.0,2434456.0,189583.0,641.0,26392.0,0.0,26392.0,9588.0,2954824.0
2,20100101,18,"ОАО КБ ""Стройкредит""",0.0,518299.0,4134606.0,8284129.0,41869.0,1484656.0,2649950.0,60642.0,1622.0,6519983.0,1701882.0,1020191.0,2550531.0,10523.0,6151173.0,2399101.0,2771.0,0.0,530918.0,-530918.0,12619.0,9405410.0
3,20100101,21,ПРИМОРСКИЙ ТЕРРИТОРИАЛЬНЫЙ,8574.0,0.0,151319.0,292503.0,878.0,97277.0,54042.0,0.0,0.0,49462.0,243041.0,21896.0,10944.0,0.0,50938.0,239624.0,1082.0,7199.0,NaN,NaN,1375.0,402204.0
4,20100101,23,АО Банк АВБ,0.0,65139.0,2654339.0,1974186.0,1091926.0,2281616.0,372723.0,6230.0,6916.0,1352310.0,608730.0,1579950.0,158720.0,16372.0,1362143.0,923421.0,1082101.0,0.0,93851.0,-93851.0,28712.0,2964819.0


Финальный датасет по отчету о прибылях и убытках в ТЫС. РУБ.

In [234]:
data_PnL_final = data_PnL[['DATE', 'REGN', 'BANK',
        'ВСЕГО Процентные доходы','Комиссионные доходы', 'Операционные доходы', 
        'Процентные расходы', 'Операционные расходы',
            
        'Корректировки, увеличивающие процентные доходы',
        'Корректировки, уменьшающие процентные расходы',
        'Доходы от восстановления (уменьшения) резервов на возможные потери',
        'Премии, уменьшающие процентные расходы',
        
        'Доходы (кроме процентных) от операций с предоставленными кредитами и прочими размещенными средствами',
        'Доходы (кроме процентных) от операций с приобретенными долговыми ценными бумагами',
        'Доходы от операций с приобретенными долевыми ценными бумагами и цифровыми финансовыми активами',
        'Доходы от операций с полученными кредитами, а также с прочими привлеченными средствами и выпущенными долговыми ценными бумагами',
        'Доходы от операций с производными финансовыми инструментами, от операций хеджирования и от операций с цифровыми финансовыми активами',
        'Комиссионные и аналогичные доходы', 
        
        'Прибыль до налогообложения',
        'Убыток до налогообложения',
        'Налог на прибыль',
        'Прибыль после налогообложения',
        'Убыток после налогообложения', 
        'Финансовый результат',
        ]]

data_PnL_final['Процентные доходы'] = data_PnL_final['ВСЕГО Процентные доходы'] - data_PnL_final['Комиссионные доходы']
col = data_PnL_final.pop('Процентные доходы')
data_PnL_final.insert(3, 'Процентные доходы', col)
data_PnL_final.drop('ВСЕГО Процентные доходы', axis=1, inplace=True)
print(data_PnL_final.shape)
data_PnL_final.head()

(36814, 24)


,DATE,REGN,BANK,Процентные доходы,Комиссионные доходы,Операционные доходы,Процентные расходы,Операционные расходы,"Корректировки, увеличивающие процентные доходы","Корректировки, уменьшающие процентные расходы",Доходы от восстановления (уменьшения) резервов на возможные потери,"Премии, уменьшающие процентные расходы",Доходы (кроме процентных) от операций с предоставленными кредитами и прочими размещенными средствами,Доходы (кроме процентных) от операций с приобретенными долговыми ценными бумагами,Доходы от операций с приобретенными долевыми ценными бумагами и цифровыми финансовыми активами,"Доходы от операций с полученными кредитами, а также с прочими привлеченными средствами и выпущенными долговыми ценными бумагами","Доходы от операций с производными финансовыми инструментами, от операций хеджирования и от операций с цифровыми финансовыми активами",Комиссионные и аналогичные доходы,Прибыль до налогообложения,Убыток до налогообложения,Налог на прибыль,Прибыль после налогообложения,Убыток после налогообложения,Финансовый результат
0,20100101,1,ЮНИКРЕДИТ БАНК,44723863.0,22682999.0,676818717.0,683892.0,685214630.0,1226587.0,29016.0,550181541.0,125381573.0,29778064.0,21184997.0,1268204.0,547702745.0,127289615.0,1674774.0,7057006.0,0.0,1257532.0,5799474.0,0.0,5799474.0
1,20100101,5,"Банк ""Кредит-Москва"" (ПАО)",422557.0,405873.0,2744820.0,4727.0,2954824.0,153676.0,75.0,2453204.0,137865.0,256249.0,330283.0,31761.0,2434456.0,189583.0,641.0,35980.0,0.0,9588.0,26392.0,0.0,26392.0
2,20100101,18,"ОАО КБ ""Стройкредит""",1484656.0,2649950.0,8284129.0,41869.0,9405410.0,60642.0,1622.0,6519983.0,1701882.0,1020191.0,2550531.0,10523.0,6151173.0,2399101.0,2771.0,0.0,518299.0,12619.0,0.0,530918.0,-530918.0
3,20100101,21,ПРИМОРСКИЙ ТЕРРИТОРИАЛЬНЫЙ,97277.0,54042.0,292503.0,878.0,402204.0,0.0,0.0,49462.0,243041.0,21896.0,10944.0,0.0,50938.0,239624.0,1082.0,8574.0,0.0,1375.0,7199.0,NaN,NaN
4,20100101,23,АО Банк АВБ,2281616.0,372723.0,1974186.0,1091926.0,2964819.0,6230.0,6916.0,1352310.0,608730.0,1579950.0,158720.0,16372.0,1362143.0,923421.0,1082101.0,0.0,65139.0,28712.0,0.0,93851.0,-93851.0


In [236]:
data_PnL_final.to_csv(r'C:\Users\annaz\Documents\Диплом\Промежуточные датафреймы\data_PnL_final.csv', index=False)

In [238]:
data_PnL_final = pd.read_csv(r'C:\Users\annaz\Documents\Диплом\Промежуточные датафреймы\data_PnL_final.csv')

In [240]:
pd.reset_option('display.max_columns')
data_PnL_final.isnull().sum()

DATE                                                                                                                                       0
REGN                                                                                                                                       0
BANK                                                                                                                                       0
Процентные доходы                                                                                                                        910
Комиссионные доходы                                                                                                                      204
Операционные доходы                                                                                                                      758
Процентные расходы                                                                                                                       780
Операционные 

# ЦИКЛ для баланса (ежеквартально), 101

## Отбор строк отчета ББ
Списки подходящих строк отчета для сбора показателей: кредитный портфель, депозиты, собствепнный капитал, финансовый результат соответственно.
Нужные номера строк бухгалтерского баланса были отобраны вручную в соответствии с Положением Банка России от 24.11.2022 N 809-П (ред. от 02.11.2024) "О Плане счетов бухгалтерского учета для кредитных организаций и порядке его применения":

https://www.consultant.ru/document/cons_doc_LAW_436264/

In [242]:
credit_num = ['441', '442', '443', '444', '445', '446', '447', '448', '449', '450', '451', '452', '45.1', '45.0', '453', '454', '455', '45.2', '456', '457', 
              '458', '460', '461', '462', '463', '464', '465', '466', '467', '468', '469', '470', '471', '472', '473', '47.1']

deposits_num = ['410', '411', '412', '413', '414', '415', '416', '417', '418', '419', '420', '421', '42.1', '422', '423', '42.2', '424', '425', '426',
               '427', '428', '429', '430', '431', '432', '433', '434', '435', '436', '437', '438', '43.1', '439', '440']

capital_num = ['102', '105', '106', '107', '108', '109', '111', '114']  ## 111 - выплаченные дивиденды (Дивиденды (распределение части прибыли между участниками))

fin_result_num = ['706', '707', '708']

npl_num = ['458']

reserves_num = ['615']

Заранее напишем функцию для расчета LIABILITIES, т.к. LIABILITIES = ПАССИВЫ - CAPITAL - FIN_RESULT.
Если CAPITAL или FIN_RESULT будут отсутсвовать в данных (NaN), то тогда LIABILITIES тоже будут считаться как NaN, но это не корректно, ведь тогда разумно предположить, что  LIABILITIES равны просто ПАССИВАМ (или ПАССИВЫ - CAPITAL / ПАССИВЫ - FIN_RESULT). 

In [150]:
# def calculate_liabilities(row):
#     if np.isnan(row['CAPITAL']): 
#         return row['CAPITALandLIABILITIES']
#     else:
#         return row['CAPITALandLIABILITIES'] - row['CAPITAL'] 

ПОМНИ! Что здесь NaN заменялись на нули, чтобы рассчитать LIABILITIES, а нули заменились на 1, чтобы сохраниить истинные нули:

In [244]:
way_101 = ("C:/Users/annaz/Documents/Диплом/Форма 101/")

file_paths = []
count = 0
all_dfs = []

for path, dirs, files in os.walk(way_101):
    for file in files:
        if file.lower().endswith('b1.dbf'):
            file_path = os.path.join(path, file)
            print(file)
            count+=1   
            
            # Скачиваем файл
            dbf = DBF(file_path, encoding='cp866')  
            df = pd.DataFrame(iter(dbf))
            df = df[df.NUM_SC != 'ITGAP']
            df_A = df[df.PLAN == 'А']

            # Создаем пустой датафрейм со всеми уникальными REGN банков за один период, в который будем записывать нужные нам показатели для каждой папки 
            empty = pd.DataFrame()
            empty['REGN'] = df_A['REGN'].unique()

            # Посчитаем активы для одной папки по каждому банку 
            assets_sum = df_A[df_A['A_P'] == "1"].groupby('REGN')['IITG'].sum().reset_index()
            empty = pd.merge(empty, assets_sum, on='REGN', how='left')
            empty = empty.rename(columns={'IITG': 'ASSETS'})

            # Посчитаем пассивы для одной папки по каждому банку 
            passive_sum = df_A[df_A['A_P'] == "2"].groupby('REGN')['IITG'].sum().reset_index()
            empty = pd.merge(empty, passive_sum, on='REGN', how='left')
            empty = empty.rename(columns={'IITG': 'CAPITALandLIABILITIES'})
            display(empty.ASSETS.sum() - empty.CAPITALandLIABILITIES.sum())

            # Посчитаем кредитный портфель каждого банка по заранее определенным номерам колокни NUM_SC. 
            # Учитываем, что мы определили только 3х значные номера, а в начале 2010-х гг. использовалось 5-ти значное исчисление колонки NUM_SC
            # а нам подходят все строки, в которых начало (первые 3 символа) соответсвует нашему списку credit_num
            loan_portfolio = df_A[(df_A['A_P'] == "1") & (df_A['NUM_SC'].str[:3].isin([str(x)[:3] for x in credit_num]))].groupby('REGN')['IITG'].sum().reset_index()
            # Посчитаем все депозиты банка за период:
            deposits = df_A[(df_A['A_P'] == "2") & (df_A['NUM_SC'].str[:3].isin([str(x)[:3] for x in deposits_num]))].groupby('REGN')['IITG'].sum().reset_index()
            # Посчитаем капитал
            capital = df_A[(df_A['NUM_SC'].str[:3].isin([str(x)[:3] for x in capital_num]))].groupby('REGN')['IITG'].sum().reset_index()
            # Посчитаем фин результат
            fin_result = df_A[(df_A['A_P'] == "2") & (df_A['NUM_SC'].str[:3].isin([str(x)[:3] for x in fin_result_num]))].groupby('REGN')['IITG'].sum().reset_index()
            # Посчитаем non-performing loan (NPL, Просроченная задолженность)
            npl = df_A[(df_A['A_P'] == "1") & (df_A['NUM_SC'].str[:3].isin([str(x)[:3] for x in npl_num]))].groupby('REGN')['IITG'].sum().reset_index()
            # Посчитаем резервы банка (reserves)
            reserves = df_A[(df_A['A_P'] == "2") & (df_A['NUM_SC'].str[:3].isin([str(x)[:3] for x in reserves_num]))].groupby('REGN')['IITG'].sum().reset_index()
            
            # Объединим все показатели банка за один период в один датасет:
            empty = pd.merge(empty, loan_portfolio, on='REGN', how='left')
            empty = empty.rename(columns={'IITG': 'LOAN_PORTFOLIO'})
            
            empty = pd.merge(empty, deposits, on='REGN', how='left')
            empty = empty.rename(columns={'IITG': 'DEPOSITS'})
            
            empty = pd.merge(empty, capital, on='REGN', how='left')
            empty = empty.rename(columns={'IITG': 'CAPITAL'})
            
            empty = pd.merge(empty, fin_result, on='REGN', how='left')
            empty = empty.rename(columns={'IITG': 'FIN_RESULT'})

            empty = pd.merge(empty, npl, on='REGN', how='left')
            empty = empty.rename(columns={'IITG': 'NPL'})

            empty = pd.merge(empty, reserves, on='REGN', how='left')
            empty = empty.rename(columns={'IITG': 'RESERVES'})
            
            # # Так как обязательства банка представляют собой разницу активов и капитала 
            # # (собственные средства + фин результат) банка, то можем рассчитать следующим образом:
            # empty['LIABILITIES'] = empty.apply(calculate_liabilities, axis=1)
            # Добавим дату, чтобы когда мы будем объединять все папки за каждый период, понимать на какую дату данный показатель
            empty['DT'] = file_path.split('-')[1][:8]
            all_dfs.append(empty)
            

print(count)
# Объединяем все фатафреймы за один период в один большой дф
df1 = pd.concat(all_dfs, ignore_index=True)
df1 = df1.rename(columns={'DT':'DATE'})
df1 = df1[['DATE', 'REGN', 'ASSETS', 'CAPITALandLIABILITIES', 'CAPITAL', 'FIN_RESULT', 'LOAN_PORTFOLIO', 'DEPOSITS', 'NPL', 'RESERVES']]
df1.to_csv(r'C:\Users\annaz\Documents\Диплом\Промежуточные датафреймы\df1.csv', index=False)
df1

122009B1.DBF


0.0

032010B1.DBF


0.0

062010B1.DBF


...

0.0

122024B1.dbf


0.0

56


,DATE,REGN,ASSETS,CAPITALandLIABILITIES,CAPITAL,FIN_RESULT,LOAN_PORTFOLIO,DEPOSITS,NPL,RESERVES
0,20100101,1,1.342929e+09,1.342929e+09,50330623.0,744909471.0,350810804.0,212786050.0,19964057.0,NaN
1,20100101,5,9.741847e+06,9.741847e+06,534742.0,3577977.0,1520709.0,3212587.0,70478.0,NaN
2,20100101,18,3.087389e+07,3.087389e+07,2792492.0,12460604.0,4657085.0,6088796.0,933364.0,NaN
3,20100101,21,1.200772e+06,1.200772e+06,139933.0,444700.0,514922.0,141822.0,31134.0,NaN
4,20100101,23,3.184786e+07,3.184786e+07,2515186.0,5720451.0,14960800.0,9350414.0,455984.0,NaN
...,...,...,...,...,...,...,...,...,...,...
36906,20250101,3403,3.630677e+07,3.630677e+07,7090705.0,12813571.0,11836994.0,7578704.0,2496536.0,101423.0
36907,20250101,3328,6.908505e+07,6.908505e+07,30495468.0,21416662.0,23554.0,159000.0,2105.0,NaN
36908,20250101,2996,3.412746e+06,3.412746e+06,729049.0,1568137.0,1038166.0,62747.0,72761.0,NaN
36909,20250101,3510,5.929226e+07,5.929226e+07,5748110.0,26463517.0,NaN,NaN,NaN,57611.0


In [29]:
df1 = pd.read_csv(r'C:\Users\annaz\Documents\Диплом\Промежуточные датафреймы\df1.csv')

In [246]:
df1.isna().sum()

DATE                         0
REGN                         0
ASSETS                       0
CAPITALandLIABILITIES        0
CAPITAL                     79
FIN_RESULT                  85
LOAN_PORTFOLIO            1807
DEPOSITS                  2512
NPL                       4651
RESERVES                 29770
dtype: int64

In [248]:
df1.shape

(36911, 10)

In [250]:
(df1 == 0).sum()

DATE                       0
REGN                       0
ASSETS                     1
CAPITALandLIABILITIES      1
CAPITAL                    1
FIN_RESULT                11
LOAN_PORTFOLIO           105
DEPOSITS                 120
NPL                      493
RESERVES                 447
dtype: int64

In [59]:
## Переведем отчет в МЛН. РУБ.
# data_BB = df1
# numeric_columns = data_BB.select_dtypes(include=['int64', 'float64']).columns 
# numeric_columns = numeric_columns.drop('REGN')
# data_BB[numeric_columns] = data_BB[numeric_columns].div(1000)
# data_BB.head()

## Добавляем название банка

In [252]:
data_BB = df1
data_BB = pd.merge(data_BB, reg_bank[['REGN', 'BANK']], on= 'REGN', how='left')
data_BB = data_BB[['DATE', 'REGN', 'BANK', 'ASSETS', 'CAPITALandLIABILITIES', 'CAPITAL', 'FIN_RESULT', 
                   'LOAN_PORTFOLIO', 'DEPOSITS', 'NPL', 'RESERVES']]
data_BB.sample(5)

,DATE,REGN,BANK,ASSETS,CAPITALandLIABILITIES,CAPITAL,FIN_RESULT,LOAN_PORTFOLIO,DEPOSITS,NPL,RESERVES
34759,20230701,3235,СЭБ БАНК,33823036.0,33823036.0,12208761.0,2165323.0,1291552.0,14988696.0,20.0,NaN
17974,20141001,1153,"ОАО ""ВКБ""",14587473.0,14587473.0,1026221.0,3631561.0,5050500.0,5617929.0,54032.0,NaN
13922,20130701,2859,"КБ ""НЕФТЯНОЙ АЛЬЯНС"" (ПАО)",11462971.0,11462971.0,838361.0,2018661.0,6913203.0,4271782.0,216044.0,NaN
23236,20160701,518,"Банк ""Первомайский"" (ПАО)",15295543.0,15295543.0,938349.0,3881309.0,6256312.0,6927745.0,1826361.0,3091.0
27110,20180101,2696,"АКБ ""ИНКАРОБАНК"" (АО)",3496637.0,3496637.0,466908.0,1929132.0,351345.0,817090.0,14651.0,10010.0


In [256]:
data_BB[(data_BB.REGN == 1481) & (data_BB.DATE == '20250101')]

,DATE,REGN,BANK,ASSETS,CAPITALandLIABILITIES,CAPITAL,FIN_RESULT,LOAN_PORTFOLIO,DEPOSITS,NPL,RESERVES
36630,20250101,1481,СБЕРБАНК РОССИИ,6.729901e+11,6.729901e+11,6.929594e+09,2.782250e+11,4.604246e+10,2.776107e+10,1.148110e+09,36704756.0


Заметим, что есть пропуски в названии банка, заполним их, погуглив названия на сайте ЦБ РФ

In [258]:
print(data_BB.isna().sum())
data_BB[data_BB['BANK'].isna()].REGN.unique()

DATE                         0
REGN                         0
BANK                         4
ASSETS                       0
CAPITALandLIABILITIES        0
CAPITAL                     79
FIN_RESULT                  85
LOAN_PORTFOLIO            1807
DEPOSITS                  2512
NPL                       4651
RESERVES                 29770
dtype: int64


array([2786, 3521], dtype=int64)

Поискав на сайте ЦБ, банк с лицензиями:

№ 2786 - "ОАО Новосибирский муниципальный банк"

№ 3521 - "НКО "ЮТИКПЭЙ" (ООО)"

№ 3551 - "РНКО Люминис (АО)"

In [260]:
bank_dict = {
    2786: 'ОАО Новосибирский муниципальный банк',
    3521: 'НКО "ЮТИКПЭЙ" (ООО)',
    # 3551: 'РНКО Люминис (АО)'
}

def fill_bank_name(df):
    if pd.isnull(df['BANK']):
        return bank_dict.get(df['REGN'], 'NaN')
    return df['BANK']

data_BB['BANK'] = data_BB.apply(fill_bank_name, axis=1)
data_BB.isna().sum()

DATE                         0
REGN                         0
BANK                         0
ASSETS                       0
CAPITALandLIABILITIES        0
CAPITAL                     79
FIN_RESULT                  85
LOAN_PORTFOLIO            1807
DEPOSITS                  2512
NPL                       4651
RESERVES                 29770
dtype: int64

In [262]:
missing_ratio = (data_BB.isna().sum() / len(data_BB))*100
missing_ratio = pd.DataFrame(round(missing_ratio, 2), columns = ['Share_of_missing, %'])
missing_ratio['Missing_count'] = data_BB.isna().sum() 
missing_ratio = missing_ratio.sort_values(by='Share_of_missing, %', ascending=True)
missing_ratio

,"Share_of_missing, %",Missing_count
DATE,0.00,0
REGN,0.00,0
BANK,0.00,0
ASSETS,0.00,0
CAPITALandLIABILITIES,0.00,0
CAPITAL,0.21,79
FIN_RESULT,0.23,85
LOAN_PORTFOLIO,4.90,1807
DEPOSITS,6.81,2512
NPL,12.60,4651


In [264]:
data_BB.to_csv(r'C:\Users\annaz\Documents\Диплом\Промежуточные датафреймы\data_BB.csv', index=False)

In [186]:
data_BB = pd.read_csv(r'C:\Users\annaz\Documents\Диплом\Промежуточные датафреймы\data_BB.csv')

In [266]:
data_BB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36911 entries, 0 to 36910
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DATE                   36911 non-null  object 
 1   REGN                   36911 non-null  int64  
 2   BANK                   36911 non-null  object 
 3   ASSETS                 36911 non-null  float64
 4   CAPITALandLIABILITIES  36911 non-null  float64
 5   CAPITAL                36832 non-null  float64
 6   FIN_RESULT             36826 non-null  float64
 7   LOAN_PORTFOLIO         35104 non-null  float64
 8   DEPOSITS               34399 non-null  float64
 9   NPL                    32260 non-null  float64
 10  RESERVES               7141 non-null   float64
dtypes: float64(8), int64(1), object(2)
memory usage: 3.1+ MB


# ЦИКЛ для нормативов (ежеквартально), 135

In [268]:
way_135 = ("C:/Users/annaz/Documents/Диплом/Форма 135/")

file_paths = []
count = 0
all_dfs = []

for path, dirs, files in os.walk(way_135):
    for file in files:
        if file.lower().endswith('_135_3.dbf'):
            file_path = os.path.join(path, file)
            print(file)
            count+=1            

            dbf = DBF(file_path, encoding='cp866')
            df = pd.DataFrame(iter(dbf))
            df['DT'] = file_path.split('-')[1][:8]
            all_dfs.append(df)
            

print(count)

df2_1 = pd.concat(all_dfs, ignore_index=True)
df2_1 = df2_1.rename(columns={'DT':'DATE', 'C2_3': 'VALUE'})
df2_1 = df2_1[['DATE', 'REGN', 'C1_3', 'VALUE']]
df2_1

032011_135_3.dbf
062011_135_3.dbf
092011_135_3.dbf
122011_135_3.dbf
032012_135_3.dbf
062012_135_3.dbf
092012_135_3.dbf
122012_135_3.dbf
032013_135_3.dbf
062013_135_3.dbf
092013_135_3.dbf
122013_135_3.dbf
032014_135_3.dbf
062014_135_3.dbf
092014_135_3.dbf
122014_135_3.dbf
032015_135_3.dbf
062015_135_3.dbf
092015_135_3.dbf
122015_135_3.dbf
032016_135_3.dbf
062016_135_3.dbf
092016_135_3.dbf
122016_135_3.dbf
032017_135_3.dbf
062017_135_3.dbf
092017_135_3.dbf
122017_135_3.dbf
032018_135_3.dbf
062018_135_3.dbf
092018_135_3.dbf
122018_135_3.dbf
032019_135_3.dbf
062019_135_3.dbf
092019_135_3.dbf
122019_135_3.dbf
032020_135_3.dbf
062020_135_3.DBF
092020_135_3.DBF
122020_135_3.DBF
032021_135_3.DBF
062021_135_3.DBF
092021_135_3.DBF
122021_135_3.DBF
062023_135_3.dbf
092023_135_3.dbf
122023_135_3.dbf
032024_135_3.dbf
062024_135_3.dbf
092024_135_3.dbf
122024_135_3.dbf
51


C:\Users\annaz\AppData\Local\Temp\ipykernel_8044\1236580373.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df2_1 = pd.concat(all_dfs, ignore_index=True)


,DATE,REGN,C1_3,VALUE
0,20110401,1,Н1,14.970
1,20110401,1,Н2,72.110
2,20110401,1,Н3,66.860
3,20110401,1,Н4,97.600
4,20110401,1,Н7,135.330
...,...,...,...,...
265294,20250101,2551,Н1.1,9.241
265295,20250101,2551,Н2,47.798
265296,20250101,567,Н1.1,11.044
265297,20250101,3388,Н1.2,13.124


In [270]:
df2_1.C1_3.unique()

array(['Н1', 'Н2', 'Н3', 'Н4', 'Н7', 'Н9.1', 'Н10.1', 'Н12', 'Н15', 'Н16',
       'Н16.1', 'Н16.2', 'Т1', 'Т2', 'Н17', 'Н18', 'Н19', 'Н6', 'Н1.1',
       'Н15.1', 'Н1.0', 'Н1.2', 'Н1.3', 'Н1.4'], dtype=object)

In [272]:
df2_1[(df2_1.C1_3 == 'Н2') & (df2_1.VALUE == 126.50)]

,DATE,REGN,C1_3,VALUE
4006,20110401,2562,Н2,126.5
18787,20111001,2557,Н2,126.5
58892,20130401,588,Н2,126.5


In [274]:
df2_1[(df2_1.REGN == 323) & (df2_1.C1_3 == 'Н2')]

,DATE,REGN,C1_3,VALUE
319,20110401,323,Н2,60.890
7726,20110701,323,Н2,57.080
15127,20111001,323,Н2,64.860
22462,20120101,323,Н2,68.670
29721,20120401,323,Н2,80.760
36991,20120701,323,Н2,69.000
44180,20121001,323,Н2,66.270
51356,20130101,323,Н2,54.870
58506,20130401,323,Н2,85.650
65647,20130701,323,Н2,73.660


Пришлось сделать второй цикл, чтобы не заморачиваться с названиями колонок в файлах 2010 г.

In [276]:
way_135_2 = ("C:/Users/annaz/Documents/Диплом/Форма 135 2010-2014/")

file_paths = []
count = 0
all_dfs = []

for path, dirs, files in os.walk(way_135_2):
    for file in files:
        if file.lower().endswith('_135_3.dbf'):
            file_path = os.path.join(path, file)
            print(file)
            count+=1            

            dbf = DBF(file_path, encoding='cp866')
            df = pd.DataFrame(iter(dbf))
            df['DT'] = file_path.split('135-')[1][:8]
            all_dfs.append(df)
            

print(count)

df2_2 = pd.concat(all_dfs, ignore_index=True)
df2_2 = df2_2.rename(columns={'DT':'DATE', 'C3_3' :'VALUE'})
df2_2 = df2_2[['DATE', 'REGN', 'C2_3', 'VALUE']]
df2_2

062010_135_3.dbf
092010_135_3.dbf
122010_135_3.dbf
3


,DATE,REGN,C2_3,VALUE
0,20100701,384,Н12,0.1
1,20100701,384,Н12,0.1
2,20100701,384,Н12,0.1
3,20100701,384,Н12,0.1
4,20100701,384,Н12,0.1
...,...,...,...,...
393,20110101,2749,Н16.1,0.0
394,20110101,2749,Н16.1,0.0
395,20110101,2756,Н3,48.8
396,20110101,2793,Н2,12.5


In [278]:
df2_2 = df2_2.rename(columns={'C2_3':'C1_3'})
df2_2.C1_3.unique()

array(['Н12', 'Н15', 'Н16.1', 'Н10.1', 'Н1', 'Н7', 'Н2', 'Н3', 'Н4'],
      dtype=object)

In [304]:
df2_2[df2_2.C1_3 == 'Н1']

,DATE,REGN,C1_3,VALUE
177,20101001,2179,Н1,7.0
178,20101001,2179,Н1,7.0
179,20101001,2179,Н1,7.0
334,20110101,2447,Н1,8.1
335,20110101,2447,Н1,8.1
336,20110101,2447,Н1,7.8
337,20110101,2447,Н1,7.8
338,20110101,2447,Н1,7.8
339,20110101,2447,Н1,7.6
340,20110101,2447,Н1,7.8


In [306]:
df2 = pd.concat([df2_1, df2_2])
df2.sample(5)

,DATE,REGN,C1_3,VALUE
215784,20190101,3536,Н16,0.00
35367,20120401,3235,Н3,57.53
62447,20130401,2705,Н10.1,0.67
68948,20130701,2462,Н2,115.98
211473,20181001,3524,Н16.2,0.00


In [308]:
df2.to_csv(r'C:\Users\annaz\Documents\Диплом\Промежуточные датафреймы\df2.csv', index=False)

## Панель

In [284]:
df2 = pd.read_csv(r'C:\Users\annaz\Documents\Диплом\Промежуточные датафреймы\df2.csv')

In [310]:
pivot_H = df2.pivot_table(index=['DATE', 'REGN'], columns='C1_3', values='VALUE', aggfunc='sum')
pivot_H = pivot_H.reset_index()
pivot_H.sample(5)

C1_3,DATE,REGN,Н1,Н1.0,Н1.1,Н1.2,Н1.3,Н1.4,Н10.1,Н12,...,Н18,Н19,Н2,Н3,Н4,Н6,Н7,Н9.1,Т1,Т2
29863,20231001,2876,NaN,193.185,NaN,193.185,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,163.129,NaN,NaN,NaN,NaN,NaN,NaN
28941,20220101,356,NaN,25.282,NaN,12.848,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,135.564,NaN,NaN,NaN,NaN,NaN,NaN
20881,20170701,2764,NaN,10.908,10.721,10.721,NaN,NaN,0.507,0.0,...,NaN,NaN,364.731,398.179,23.418,NaN,289.813,0.0,NaN,NaN
31465,20250101,1115,NaN,27.543,NaN,14.899,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,182.638,NaN,NaN,NaN,NaN,NaN,NaN
6138,20121001,2743,75.95,NaN,NaN,NaN,NaN,NaN,0.000,0.0,...,NaN,NaN,218.000,225.490,57.270,NaN,33.310,0.0,NaN,NaN


## Выбираем нормативы

H1.0 - Норматив достаточности совокупного капитала, %

H1.1 - Норматив достаточности базового капитала, %

H1.2 - Норматив достаточности основного капитала, %

H2	- Норматив мгновенной ликвидности, %

H3	- Норматив текущей ликвидности, %

H4	- Норматив долгосрочной ликвидности, %

Н7 - Максимальный размер крупных кредитных рисков, %

In [288]:
pivot_H.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31731 entries, 0 to 31730
Data columns (total 26 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DATE    31731 non-null  int64  
 1   REGN    31731 non-null  int64  
 2   Н1      10957 non-null  float64
 3   Н1.0    20339 non-null  float64
 4   Н1.1    16868 non-null  float64
 5   Н1.2    19130 non-null  float64
 6   Н1.3    302 non-null    float64
 7   Н1.4    4819 non-null   float64
 8   Н10.1   24981 non-null  float64
 9   Н12     26517 non-null  float64
 10  Н15     1728 non-null   float64
 11  Н15.1   312 non-null    float64
 12  Н16     1245 non-null   float64
 13  Н16.1   1591 non-null   float64
 14  Н16.2   1569 non-null   float64
 15  Н17     24 non-null     float64
 16  Н18     991 non-null    float64
 17  Н19     24 non-null     float64
 18  Н2      27808 non-null  float64
 19  Н3      30132 non-null  float64
 20  Н4      27765 non-null  float64
 21  Н6      5 non-null      float64
 22

In [315]:
missing_ratio = (pivot_H.isna().sum()  / len(pivot_H))*100
missing_ratio = pd.DataFrame(round(missing_ratio, 2), columns = ['Share_of_missing, %'])
missing_ratio['Missing_count'] = pivot_H.isna().sum() 
missing_ratio = missing_ratio.sort_values(by='Share_of_missing, %', ascending=True)
missing_ratio.head(15)

,"Share_of_missing, %",Missing_count
C1_3,,
DATE,0.00,0
REGN,0.00,0
Н1.0,1.37,435
Н3,5.04,1599
Н2,12.36,3923
Н4,12.50,3966
Н12,16.43,5214
Н7,17.76,5634
Н10.1,21.27,6750


Оказывается норматив Н1.0 полностью заменил норматив Н1 только в 2020 г., а ранее, особенно до 2015 г. банки публиковали либо Н1 и Н1.0, поэтому считаю целесообразным заполнить пропуски в показатели Н1.0 значениями Н1, если такие имеются.

In [313]:
pivot_H['Н1.0'] = pivot_H['Н1.0'].fillna(pivot_H['Н1'])

## Добавляем название банка

In [317]:
data_norm = pd.merge(pivot_H, reg_bank[['REGN', 'BANK']], on= 'REGN', how='left')
data_norm = data_norm[['DATE', 'REGN', 'BANK', 'Н1.0', 'Н1.1', 'Н1.2', 'Н2', 'Н3', 'Н4', 'Н7']]
print(data_norm.isna().sum())
data_norm.sample(5)

DATE        0
REGN        0
BANK       43
Н1.0      435
Н1.1    14863
Н1.2    12601
Н2       3923
Н3       1599
Н4       3966
Н7       5634
dtype: int64


,DATE,REGN,BANK,Н1.0,Н1.1,Н1.2,Н2,Н3,Н4,Н7
25101,20190701,3245,ФИНСТАР,13.632,13.632,13.632,43.142,134.411,8.958,376.908
562,20110401,2659,АЛТАЙКАПИТАЛБАНК,21.980,NaN,NaN,38.290,72.650,56.190,205.610
15233,20150701,485,ЧЕЛИНДБАНК,17.470,11.690,11.690,396.670,258.830,31.780,101.750
2928,20120101,600,ИНТЕРПРОГРЕССБАНК,13.800,NaN,NaN,66.010,87.970,38.630,463.980
5455,20120701,3303,АЗИЯ-ИНВЕСТ БАНК,24.150,NaN,NaN,70.090,90.330,8.530,345.690


In [319]:
data_norm[data_norm.BANK == 'ПАО "БИНБАНК"']

,DATE,REGN,BANK,Н1.0,Н1.1,Н1.2,Н2,Н3,Н4,Н7
58,20110401,323,"ПАО ""БИНБАНК""",14.530,NaN,NaN,60.890,77.680,102.360,195.930
526,20110401,2562,"ПАО ""БИНБАНК""",11.860,NaN,NaN,126.500,141.450,84.850,471.280
1001,20110701,323,"ПАО ""БИНБАНК""",13.430,NaN,NaN,57.080,64.140,98.920,192.800
1467,20110701,2562,"ПАО ""БИНБАНК""",11.960,NaN,NaN,88.460,113.570,73.630,517.900
1941,20111001,323,"ПАО ""БИНБАНК""",12.710,NaN,NaN,64.860,62.050,87.750,199.210
2405,20111001,2562,"ПАО ""БИНБАНК""",12.480,NaN,NaN,77.600,108.330,69.540,479.300
2872,20120101,323,"ПАО ""БИНБАНК""",12.830,NaN,NaN,68.670,92.610,65.400,201.700
3329,20120101,2562,"ПАО ""БИНБАНК""",12.300,NaN,NaN,102.980,109.280,57.990,475.440
3792,20120401,323,"ПАО ""БИНБАНК""",11.730,NaN,NaN,80.760,77.830,58.740,217.230
4250,20120401,2562,"ПАО ""БИНБАНК""",12.460,NaN,NaN,99.300,147.850,54.030,451.380


Опять замечаем, что не подтянулось 7 названиев банков, подтянем их вручную

https://www.cbr.ru/banking_sector/credit/colistfo/

In [321]:
data_norm[data_norm['BANK'].isna()].REGN.unique()  

array([1202, 1909, 2786, 2787, 2836, 3521], dtype=int64)

Поискав на сайте ЦБ, банк с лицензиями:

№ 1202 - ЗАО "Башинвестбанк"

№ 1909 - ООО Коммерческий банк «Русско-Ингушский»

№ 2786 - "ОАО Новосибирский муниципальный банк"

№ 2787 -  САМАРА-БАНК

№ 2836 - АБ "БРП" (ОАО)

№ 3521 - "НКО "ЮТИКПЭЙ" (ООО)"

№ 3551 - "РНКО Люминис (АО)"

In [326]:
bank_dict = {
    1202: 'ЗАО "Башинвестбанк"',
    1909: 'ООО Коммерческий банк «Русско-Ингушский»',
    2786: 'ОАО Новосибирский муниципальный банк',
    2787: 'САМАРА-БАНК',
    2836: 'АБ "БРП" (ОАО)',
    3521: 'НКО "ЮТИКПЭЙ" (ООО)',
    # 3551: 'РНКО Люминис (АО)'
}

data_norm['BANK'] = data_norm.apply(fill_bank_name, axis=1)
data_norm.isna().sum()

DATE        0
REGN        0
BANK        0
Н1.0      435
Н1.1    14863
Н1.2    12601
Н2       3923
Н3       1599
Н4       3966
Н7       5634
dtype: int64

In [328]:
data_norm.to_csv(r'C:\Users\annaz\Documents\Диплом\Промежуточные датафреймы\data_norm.csv', index=False)

In [109]:
data_norm = pd.read_csv(r'C:\Users\annaz\Documents\Диплом\Промежуточные датафреймы\data_norm.csv')

# ЦИКЛ для капитала (ежеквартально), 123, 134

Так как до 2014 года отчетность о капитале публиковалась в форме 134, то будем отдельно формировать два цикла для формы 123 (данные о капитале с 2014 по настоящий момент) и для формы 134 (данные о капитале с 2010 по 2014 г.)

In [330]:
way_123 = ("C:/Users/annaz/Documents/Диплом/Форма 123/")

file_paths = []
count = 0
all_dfs = []

for path, dirs, files in os.walk(way_123):
    for file in files:
        if file.lower().endswith('_123d.dbf'):
            file_path = os.path.join(path, file)
            print(file)
            count+=1            

            dbf = DBF(file_path, encoding='cp866')
            df = pd.DataFrame(iter(dbf)) 
            df['DT'] = file_path.split('-')[1][:8]
            all_dfs.append(df)
            

print(count)

df3_0 = pd.concat(all_dfs, ignore_index=True)
df3_0 = df3_0.rename(columns={'DT':'DATE', 'C3': 'VALUE'})
df3_0 = df3_0[['DATE', 'REGN', 'C1', 'VALUE']]
df3_0

032014_123D.DBF
062014_123D.DBF
092014_123D.DBF
122014_123D.DBF
032015_123D.DBF
062015_123D.DBF
092015_123D.DBF
122015_123D.DBF
032016_123D.DBF
062016_123D.DBF
092016_123D.DBF
122016_123D.DBF
032017_123D.DBF
062017_123D.DBF
092017_123D.DBF
122017_123D.DBF
032018_123D.DBF
062018_123D.DBF
092018_123D.DBF
122018_123D.DBF
032019_123D.DBF
062019_123D.DBF
092019_123D.DBF
122019_123D.DBF
032020_123D.DBF
062020_123D.DBF
092020_123D.DBF
122020_123D.DBF
032021_123D.DBF
29


,DATE,REGN,C1,VALUE
0,20140401,1,000,132146432
1,20140401,1,100.1,40438324
2,20140401,1,100.1.1,40438324
3,20140401,1,100.1.2,0
4,20140401,1,100.1.3,0
...,...,...,...,...
2235645,20210401,990,202.1,0
2235646,20210401,990,202.2,0
2235647,20210401,990,202.3,0
2235648,20210401,990,202.3.1,42126


In [332]:
way_134 = (r"C:\Users\annaz\Documents\Диплом\Форма 134")

file_paths = []
count = 0
all_dfs = []

for path, dirs, files in os.walk(way_134):
    for file in files:
        if file.lower().endswith('_134d.dbf'):
            file_path = os.path.join(path, file)
            print(file)
            count+=1            

            dbf = DBF(file_path, encoding='cp866')
            df = pd.DataFrame(iter(dbf)) 
            df['DT'] = file_path.split('-')[1][:8]
            all_dfs.append(df)
            

print(count)

df3_1 = pd.concat(all_dfs, ignore_index=True)
df3_1 = df3_1.rename(columns={'DT':'DATE', 'C3': 'VALUE'})
df3_1 = df3_1[['DATE', 'REGN', 'C1', 'VALUE']]
df3_1

062010_134D.dbf
092010_134D.dbf
122010_134D.dbf
032011_134D.dbf
062011_134D.dbf
092011_134D.dbf
122011_134D.dbf
032012_134D.dbf
062012_134D.dbf
092012_134D.dbf
122012_134D.dbf
032013_134D.dbf
062013_134D.dbf
092013_134D.dbf
122013_134D.dbf
15


,DATE,REGN,C1,VALUE
0,20100701,1,000,66363861
1,20100701,1,101,23064358
2,20100701,1,102,7837
3,20100701,1,103,2161823
4,20100701,1,104,0
...,...,...,...,...
651457,20140101,3515,302,0
651458,20140101,3515,400,4240071
651459,20140101,3515,501,0
651460,20140101,3515,502,0


In [334]:
df3_1.DATE.unique()

array(['20100701', '20101001', '20110101', '20110401', '20110701',
       '20111001', '20120101', '20120401', '20120701', '20121001',
       '20130101', '20130401', '20130701', '20131001', '20140101'],
      dtype=object)

Объединим два датасета

In [336]:
df3 = pd.concat([df3_0, df3_1])

Так как с 2021 г. форма 123 хранится в dbf с другим декодингом, который не считывается привычными encoding (cp866 и др.), было принято решение вручную объединить эти файлы в excel-файл и просто прочитать его:

In [338]:
df3_2 = pd.read_excel(r"C:\Users\annaz\Documents\Диплом\Форма 123 072021-2025\123_from_01072021.xlsx")
df3_2['C1'] = df3_2['C1'].astype(str)
df3_2['C1'] = df3_2['C1'].str.split('_').str[0]  ## работает для файла xlsx, а не csv
df3_2.head(5)

,DATE,REGN,C1,VALUE
0,20210701,1,200.6.4,0
1,20210701,1,100.5.7,0
2,20210701,1,101.7.8.1,0
3,20210701,1,104.3.1,0
4,20210701,1,201.4.1,0


In [340]:
df3_2.DATE = df3_2.DATE.astype(str)
df3_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201269 entries, 0 to 201268
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   DATE    201269 non-null  object
 1   REGN    201269 non-null  int64 
 2   C1      201269 non-null  object
 3   VALUE   201269 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 6.1+ MB


In [342]:
df3_2.to_csv(r"C:\Users\annaz\Documents\Диплом\Форма 123 072021-2025\123_from_01072021.csv", index=False)

Объединим все три датасета:

In [344]:
df3 = pd.concat([df3, df3_2])
df3[(df3.REGN == 1481) & (df3.DATE == '20250101')]

,DATE,REGN,C1,VALUE
200004,20250101,1481,000,6941258181
200005,20250101,1481,102,5933876391
200006,20250101,1481,105,150000000
200007,20250101,1481,203,857381790


In [346]:
df3.to_csv(r'C:\Users\annaz\Documents\Диплом\Промежуточные датафреймы\df3.csv', index=False)

In [348]:
df3.isna().sum()

DATE     0
REGN     0
C1       0
VALUE    0
dtype: int64

## Панель
Будем брать только строку "000", так как она отражает Итого Собственные средства банка.

In [3]:
df3 = pd.read_csv(r'C:\Users\annaz\Documents\Диплом\Промежуточные датафреймы\df3.csv')

In [350]:
pivot_cap = df3.pivot_table(index=['DATE', 'REGN'], columns='C1', values='VALUE', aggfunc='sum')
pivot_cap = pivot_cap.reset_index()

data_equity = pivot_cap[['DATE', 'REGN', '000']]
data_equity = data_equity.rename(columns={'000': 'EQUITY'})
data_equity

C1,DATE,REGN,EQUITY
0,20100701,1,66363861.0
1,20100701,2,511548.0
2,20100701,52,723674.0
3,20100701,53,1005700.0
4,20100701,55,554615.0
...,...,...,...
33863,20250101,3545,21068856.0
33864,20250101,3546,413016.0
33865,20250101,3547,461742.0
33866,20250101,3548,133721.0


## Добавляем название банков

In [352]:
data_equity = pd.merge(data_equity, reg_bank[['REGN', 'BANK']], on= 'REGN', how='left')
data_equity = data_equity[['DATE', 'REGN', 'BANK', 'EQUITY']]
print(data_equity.isna().sum())
data_equity.sample(5)

DATE       0
REGN       0
BANK      49
EQUITY     0
dtype: int64


,DATE,REGN,BANK,EQUITY
491,20100701,2960,"ООО КБ ""Славянский кредит""",1439082.0
12437,20131001,3504,"АО ""СМБ-Банк""",255835.0
6773,20120401,3065,"Банк ""ВПБ"" (АО)",896599.0
5261,20120101,604,ПЕРВЫЙ ИНВЕСТИЦИОННЫЙ БАНК,544993.0
31812,20231001,783,РОЯЛ КРЕДИТ БАНК,858351.0


In [354]:
data_equity[data_equity.BANK.isna()].REGN.unique()

array([1202, 1909, 2786, 2787, 2836, 3521], dtype=int64)

In [356]:
data_equity['BANK'] = data_equity.apply(fill_bank_name, axis=1)
data_equity.isna().sum()

DATE      0
REGN      0
BANK      0
EQUITY    0
dtype: int64

In [358]:
data_equity[(data_equity.REGN == 1481) & (data_equity.DATE == '20250101')]

,DATE,REGN,BANK,EQUITY
33616,20250101,1481,СБЕРБАНК РОССИИ,6.941258e+09


In [360]:
data_equity.to_csv(r'C:\Users\annaz\Documents\Диплом\Промежуточные датафреймы\data_equity.csv', index=False)

In [111]:
data_equity = pd.read_csv(r'C:\Users\annaz\Documents\Диплом\Промежуточные датафреймы\data_equity.csv')

# Объединяем отчеты в одну панель

In [461]:
print(data_PnL_final.shape)
data_PnL_final['DATE'] = data_PnL_final['DATE'].astype(str)
print(data_PnL_final.DATE.info())
data_PnL_final.sample(5)

In [364]:
print(data_BB.shape)
data_BB['DATE'] = data_BB['DATE'].astype(str)
print(data_BB.DATE.info())
data_BB.sample(5)

(36911, 11)
<class 'pandas.core.series.Series'>
RangeIndex: 36911 entries, 0 to 36910
Series name: DATE
Non-Null Count  Dtype 
--------------  ----- 
36911 non-null  object
dtypes: object(1)
memory usage: 288.5+ KB
None


,DATE,REGN,BANK,ASSETS,CAPITALandLIABILITIES,CAPITAL,FIN_RESULT,LOAN_PORTFOLIO,DEPOSITS,NPL,RESERVES
9192,20120401,2604,"АКБ ""Сибирский Энергетический Банк"" (ЗАО)",1.250815e+06,1.250815e+06,100885.0,1.154100e+05,9.070370e+05,7.965470e+05,34457.0,NaN
3915,20110101,480,ТАТСОЦБАНК,1.766032e+06,1.766032e+06,818405.0,1.870550e+05,6.708620e+05,1.253800e+05,777.0,NaN
25606,20170401,3460,АМЕРИКЭН ЭКСПРЕСС БАНК,3.862288e+06,3.862288e+06,824212.0,1.977761e+06,9.454490e+05,NaN,0.0,NaN
21393,20151001,2494,БАНК КРЕДИТ СВИСС (МОСКВА),1.116967e+08,1.116967e+08,12784785.0,7.834876e+07,1.233450e+05,1.891800e+04,NaN,NaN
13495,20130701,1000,ВТБ,7.985728e+09,7.985728e+09,746668440.0,1.268107e+09,2.309696e+09,1.613843e+09,111771037.0,51937.0


In [366]:
print(data_norm.shape)
data_norm['DATE'] = data_norm['DATE'].astype(str)
print(data_norm.DATE.info())
data_norm.sample(5)

(31731, 10)
<class 'pandas.core.series.Series'>
RangeIndex: 31731 entries, 0 to 31730
Series name: DATE
Non-Null Count  Dtype 
--------------  ----- 
31731 non-null  object
dtypes: object(1)
memory usage: 248.0+ KB
None


,DATE,REGN,BANK,Н1.0,Н1.1,Н1.2,Н2,Н3,Н4,Н7
14627,20150401,1732,ИК БАНК,31.530,26.300,26.300,87.750,104.030,43.420,70.54
11981,20140701,870,"ООО ""Владпромбанк""",16.470,8.020,16.400,189.990,165.800,75.970,223.66
13796,20150101,1614,"АО ""Заубер Банк""",36.700,30.440,30.440,90.860,137.050,26.910,90.93
26276,20200401,2168,ДРАЙВ КЛИК БАНК,14.054,10.803,10.803,60.168,74.812,105.165,0.00
10729,20140101,2992,"КБ ""БМБ"" (ООО)",12.150,NaN,NaN,59.400,83.860,86.210,409.06


In [368]:
print(data_equity.shape)
data_equity['DATE'] = data_equity['DATE'].astype(str)
print(data_equity.DATE.info())
data_equity.sample(5)

(33868, 4)
<class 'pandas.core.series.Series'>
RangeIndex: 33868 entries, 0 to 33867
Series name: DATE
Non-Null Count  Dtype 
--------------  ----- 
33868 non-null  object
dtypes: object(1)
memory usage: 264.7+ KB
None


,DATE,REGN,BANK,EQUITY
14275,20140701,1810,АЗИАТСКО-ТИХООКЕАНСКИЙ БАНК,13246159.0
4323,20111001,574,ЗЕМЕЛЬНЫЙ,552966.0
9272,20130101,2402,ЕВРОФИНАНС МОСНАРБАНК,12725235.0
8614,20121001,3120,"ПАО ""Тайм Банк""",239809.0
1518,20110101,622,"ООО КБ ""АкадемРусБанк""",122270.0


In [370]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [372]:
data_BB = data_BB.drop('BANK', axis=1)
RESULT = pd.merge(data_PnL_final, data_BB, on=['DATE', 'REGN'])
RESULT.head()

,DATE,REGN,BANK,Процентные доходы,Комиссионные доходы,Операционные доходы,Процентные расходы,Операционные расходы,"Корректировки, увеличивающие процентные доходы","Корректировки, уменьшающие процентные расходы",Доходы от восстановления (уменьшения) резервов на возможные потери,"Премии, уменьшающие процентные расходы",Доходы (кроме процентных) от операций с предоставленными кредитами и прочими размещенными средствами,Доходы (кроме процентных) от операций с приобретенными долговыми ценными бумагами,Доходы от операций с приобретенными долевыми ценными бумагами и цифровыми финансовыми активами,"Доходы от операций с полученными кредитами, а также с прочими привлеченными средствами и выпущенными долговыми ценными бумагами","Доходы от операций с производными финансовыми инструментами, от операций хеджирования и от операций с цифровыми финансовыми активами",Комиссионные и аналогичные доходы,Прибыль до налогообложения,Убыток до налогообложения,Налог на прибыль,Прибыль после налогообложения,Убыток после налогообложения,Финансовый результат,ASSETS,CAPITALandLIABILITIES,CAPITAL,FIN_RESULT,LOAN_PORTFOLIO,DEPOSITS,NPL,RESERVES
0,20100101,1,ЮНИКРЕДИТ БАНК,44723863.0,22682999.0,676818717.0,683892.0,685214630.0,1226587.0,29016.0,550181541.0,125381573.0,29778064.0,21184997.0,1268204.0,547702745.0,127289615.0,1674774.0,7057006.0,0.0,1257532.0,5799474.0,0.0,5799474.0,1.342929e+09,1.342929e+09,50330623.0,744909471.0,350810804.0,212786050.0,19964057.0,NaN
1,20100101,5,"Банк ""Кредит-Москва"" (ПАО)",422557.0,405873.0,2744820.0,4727.0,2954824.0,153676.0,75.0,2453204.0,137865.0,256249.0,330283.0,31761.0,2434456.0,189583.0,641.0,35980.0,0.0,9588.0,26392.0,0.0,26392.0,9.741847e+06,9.741847e+06,534742.0,3577977.0,1520709.0,3212587.0,70478.0,NaN
2,20100101,18,"ОАО КБ ""Стройкредит""",1484656.0,2649950.0,8284129.0,41869.0,9405410.0,60642.0,1622.0,6519983.0,1701882.0,1020191.0,2550531.0,10523.0,6151173.0,2399101.0,2771.0,0.0,518299.0,12619.0,0.0,530918.0,-530918.0,3.087389e+07,3.087389e+07,2792492.0,12460604.0,4657085.0,6088796.0,933364.0,NaN
3,20100101,21,ПРИМОРСКИЙ ТЕРРИТОРИАЛЬНЫЙ,97277.0,54042.0,292503.0,878.0,402204.0,0.0,0.0,49462.0,243041.0,21896.0,10944.0,0.0,50938.0,239624.0,1082.0,8574.0,0.0,1375.0,7199.0,NaN,NaN,1.200772e+06,1.200772e+06,139933.0,444700.0,514922.0,141822.0,31134.0,NaN
4,20100101,23,АО Банк АВБ,2281616.0,372723.0,1974186.0,1091926.0,2964819.0,6230.0,6916.0,1352310.0,608730.0,1579950.0,158720.0,16372.0,1362143.0,923421.0,1082101.0,0.0,65139.0,28712.0,0.0,93851.0,-93851.0,3.184786e+07,3.184786e+07,2515186.0,5720451.0,14960800.0,9350414.0,455984.0,NaN


In [374]:
data_norm = data_norm.drop('BANK', axis=1)
RESULT = pd.merge(RESULT, data_norm, on=['DATE', 'REGN'])
RESULT.head()

,DATE,REGN,BANK,Процентные доходы,Комиссионные доходы,Операционные доходы,Процентные расходы,Операционные расходы,"Корректировки, увеличивающие процентные доходы","Корректировки, уменьшающие процентные расходы",Доходы от восстановления (уменьшения) резервов на возможные потери,"Премии, уменьшающие процентные расходы",Доходы (кроме процентных) от операций с предоставленными кредитами и прочими размещенными средствами,Доходы (кроме процентных) от операций с приобретенными долговыми ценными бумагами,Доходы от операций с приобретенными долевыми ценными бумагами и цифровыми финансовыми активами,"Доходы от операций с полученными кредитами, а также с прочими привлеченными средствами и выпущенными долговыми ценными бумагами","Доходы от операций с производными финансовыми инструментами, от операций хеджирования и от операций с цифровыми финансовыми активами",Комиссионные и аналогичные доходы,Прибыль до налогообложения,Убыток до налогообложения,Налог на прибыль,Прибыль после налогообложения,Убыток после налогообложения,Финансовый результат,ASSETS,CAPITALandLIABILITIES,CAPITAL,FIN_RESULT,LOAN_PORTFOLIO,DEPOSITS,NPL,RESERVES,Н1.0,Н1.1,Н1.2,Н2,Н3,Н4,Н7
0,20100701,384,"НКО ""ЕРП"" (ООО)",5119.0,22925.0,5680.0,23.0,31604.0,0.0,0.0,4247.0,1433.0,293.0,1317.0,0.0,4243.0,1793.0,226.0,307.0,0.0,211.0,96.0,0.0,96.0,114968.0,114968.0,10481.0,33747.0,31006.0,5048.0,7925.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20100701,2749,"ООО РНКО ""РИБ""",178.0,2921.0,380.0,0.0,3322.0,3.0,0.0,0.0,377.0,41.0,0.0,0.0,0.0,103.0,20.0,96.0,0.0,0.0,96.0,0.0,96.0,65463.0,65463.0,8398.0,3479.0,1472.0,645.0,325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20100701,2755,НК БАНК,625463.0,172953.0,3382452.0,24984.0,3596575.0,7066.0,205.0,2086828.0,1288353.0,290570.0,157870.0,7379.0,2089324.0,1308170.0,1030.0,159807.0,0.0,23978.0,135829.0,0.0,135829.0,14382586.0,14382586.0,1450205.0,4281332.0,6770787.0,2411766.0,254799.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20101001,384,"НКО ""ЕРП"" (ООО)",6707.0,35715.0,7351.0,32.0,47045.0,0.0,0.0,5467.0,1884.0,317.0,1317.0,0.0,5445.0,4653.0,494.0,632.0,0.0,201.0,431.0,0.0,431.0,141190.0,141190.0,10481.0,49805.0,26895.0,4606.0,24085.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20101001,2179,"ОАО Банк ""ОТКРЫТИЕ""",8270906.0,25572151.0,57657914.0,160785.0,48862672.0,4865444.0,9836.0,20970301.0,31812333.0,4399212.0,30365189.0,3571507.0,21000440.0,19726379.0,253257.0,7781426.0,0.0,261767.0,7519659.0,0.0,7519659.0,197123028.0,197123028.0,17733164.0,48481306.0,54758479.0,56158479.0,3416978.0,NaN,21.0,NaN,NaN,NaN,NaN,NaN,2503.5


In [376]:
data_equity = data_equity.drop('BANK', axis=1)
RESULT = pd.merge(RESULT, data_equity, on=['DATE', 'REGN'])
RESULT.head()

,DATE,REGN,BANK,Процентные доходы,Комиссионные доходы,Операционные доходы,Процентные расходы,Операционные расходы,"Корректировки, увеличивающие процентные доходы","Корректировки, уменьшающие процентные расходы",Доходы от восстановления (уменьшения) резервов на возможные потери,"Премии, уменьшающие процентные расходы",Доходы (кроме процентных) от операций с предоставленными кредитами и прочими размещенными средствами,Доходы (кроме процентных) от операций с приобретенными долговыми ценными бумагами,Доходы от операций с приобретенными долевыми ценными бумагами и цифровыми финансовыми активами,"Доходы от операций с полученными кредитами, а также с прочими привлеченными средствами и выпущенными долговыми ценными бумагами","Доходы от операций с производными финансовыми инструментами, от операций хеджирования и от операций с цифровыми финансовыми активами",Комиссионные и аналогичные доходы,Прибыль до налогообложения,Убыток до налогообложения,Налог на прибыль,Прибыль после налогообложения,Убыток после налогообложения,Финансовый результат,ASSETS,CAPITALandLIABILITIES,CAPITAL,FIN_RESULT,LOAN_PORTFOLIO,DEPOSITS,NPL,RESERVES,Н1.0,Н1.1,Н1.2,Н2,Н3,Н4,Н7,EQUITY
0,20100701,384,"НКО ""ЕРП"" (ООО)",5119.0,22925.0,5680.0,23.0,31604.0,0.0,0.0,4247.0,1433.0,293.0,1317.0,0.0,4243.0,1793.0,226.0,307.0,0.0,211.0,96.0,0.0,96.0,114968.0,114968.0,10481.0,33747.0,31006.0,5048.0,7925.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10375.0
1,20100701,2749,"ООО РНКО ""РИБ""",178.0,2921.0,380.0,0.0,3322.0,3.0,0.0,0.0,377.0,41.0,0.0,0.0,0.0,103.0,20.0,96.0,0.0,0.0,96.0,0.0,96.0,65463.0,65463.0,8398.0,3479.0,1472.0,645.0,325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8444.0
2,20100701,2755,НК БАНК,625463.0,172953.0,3382452.0,24984.0,3596575.0,7066.0,205.0,2086828.0,1288353.0,290570.0,157870.0,7379.0,2089324.0,1308170.0,1030.0,159807.0,0.0,23978.0,135829.0,0.0,135829.0,14382586.0,14382586.0,1450205.0,4281332.0,6770787.0,2411766.0,254799.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1788056.0
3,20101001,384,"НКО ""ЕРП"" (ООО)",6707.0,35715.0,7351.0,32.0,47045.0,0.0,0.0,5467.0,1884.0,317.0,1317.0,0.0,5445.0,4653.0,494.0,632.0,0.0,201.0,431.0,0.0,431.0,141190.0,141190.0,10481.0,49805.0,26895.0,4606.0,24085.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10717.0
4,20101001,2179,"ОАО Банк ""ОТКРЫТИЕ""",8270906.0,25572151.0,57657914.0,160785.0,48862672.0,4865444.0,9836.0,20970301.0,31812333.0,4399212.0,30365189.0,3571507.0,21000440.0,19726379.0,253257.0,7781426.0,0.0,261767.0,7519659.0,0.0,7519659.0,197123028.0,197123028.0,17733164.0,48481306.0,54758479.0,56158479.0,3416978.0,NaN,21.0,NaN,NaN,NaN,NaN,NaN,2503.5,18825433.0


In [378]:
RESULT.shape

(31064, 40)

In [380]:
pd.reset_option('display.max_columns')

In [382]:
RESULT.to_csv(r'C:\Users\annaz\Documents\Диплом\Промежуточные датафреймы\RESULT.csv', index=False)

## Объединенный датасет

In [113]:
RESULT = pd.read_csv(r'C:\Users\annaz\Documents\Диплом\Промежуточные датафреймы\RESULT.csv')
print(RESULT.shape)
RESULT.sample(3)

(31064, 40)


,DATE,REGN,BANK,Процентные доходы,Комиссионные доходы,Операционные доходы,Процентные расходы,Операционные расходы,"Корректировки, увеличивающие процентные доходы","Корректировки, уменьшающие процентные расходы",...,NPL,RESERVES,Н1.0,Н1.1,Н1.2,Н2,Н3,Н4,Н7,EQUITY
5292,20120701,3206,"""Соверен Банк"" (АО)",12265.0,11434.0,60392.0,20386.0,77326.0,NaN,NaN,...,NaN,NaN,88.35,NaN,NaN,933.78,1185.93,0.00,54.49,200519.0
10142,20140101,1459,"ООО КБ ""Агросоюз""",873518.0,753448.0,2367322.0,8781.0,2908688.0,19662.0,537.0,...,11166.0,NaN,13.88,NaN,NaN,83.01,88.51,58.46,347.43,1074444.0
10895,20140401,1035,"ОАО ККБ ""КРЕДИТБАНК""",21187.0,7581.0,90380.0,18.0,103251.0,0.0,0.0,...,1666.0,NaN,27.23,24.8,24.8,115.06,102.94,10.60,212.84,232729.0


In [111]:
print(RESULT.shape)
RESULT.isna().sum()
MISSING = (RESULT.isna().sum() / len(RESULT))*100
MISSING = pd.DataFrame(round(MISSING, 2), columns = ['Share_of_missing, %'])
MISSING['Missing_count'] = RESULT.isna().sum()
MISSING = MISSING.sort_values(by='Share_of_missing, %', ascending=True)
MISSING

(31064, 40)


,"Share_of_missing, %",Missing_count
DATE,0.00,0
CAPITALandLIABILITIES,0.00,0
ASSETS,0.00,0
EQUITY,0.00,0
REGN,0.00,0
BANK,0.00,0
CAPITAL,0.25,79
FIN_RESULT,0.27,84
Прибыль до налогообложения,0.37,114
Комиссионные и аналогичные доходы,0.58,180


# Макропоказатели

In [ ]:
ans = RESULT

## RUONIA
Source, the Central Bank of Russia:

https://www.cbr.ru/hd_base/ruonia/dynamics/?UniDbQuery.Posted=True&UniDbQuery.From=11.01.2010&UniDbQuery.To=19.03.2025

In [113]:
RUONIA = pd.read_excel(r"C:\Users\annaz\Documents\Диплом\Macro data\Динамика ставки RUONIA.xlsx")
RUONIA = RUONIA.rename(columns={'ruo': 'RUONIA', 'vol': 'volume', 'T': 'Number_of_transactions','C': 'Number_of_participants'})
RUONIA.head(3)

C:\Users\annaz\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,DT,RUONIA,volume,Number_of_transactions,Number_of_participants,MinRate,Percentile25,Percentile75,MaxRate,StatusXML,DateUpdate
0,2025-03-19,21.45,360.34,51.0,19.0,19.8,21.20,21.54,21.75,0.0,2025-03-20 14:13:25.973
1,2025-03-18,21.40,412.14,47.0,19.0,19.8,21.16,21.50,21.60,0.0,2025-03-19 14:20:28.340
2,2025-03-17,21.41,412.21,48.0,16.0,21.0,21.15,21.50,21.60,0.0,2025-03-18 14:15:17.367


In [115]:
RUONIA['year'] = RUONIA['DT'].dt.year
RUONIA['year'] = RUONIA['year'].astype(str)
RUONIA['month'] = RUONIA['DT'].dt.month
RUONIA['month'] = RUONIA['month'].astype(str)

RUONIA = RUONIA.groupby(['year', 'month'])['RUONIA'].mean().reset_index()
RUONIA.sample(3)

,year,month,RUONIA
166,2023,8,10.339565
16,2011,2,2.741579
0,2010,1,3.974000


In [117]:
RUONIA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    183 non-null    object 
 1   month   183 non-null    object 
 2   RUONIA  183 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.4+ KB


In [119]:
ans = pd.merge(ans, RUONIA[['year', 'month', 'RUONIA']], on=['year', 'month'], how='left')

print(ans.shape)
ans.sample(5)

(31064, 42)


,DATE,year,month,REGN,BANK,Interest_Income,Commission_Income,Operating_Income,Other_Income,Interest_Expense,Operating_Expense,EBT,Tax,Net_Profit,ASSETS,LIABILITIES,EQUITY,LOAN_PORTFOLIO,DEPOSITS,Н1.0,Н1.1,Н1.2,Н2,Н3,Н4,ROE,ROA,NIM,CIR,CAR,LTD,Bank_Size,Revenue,Leverage,Net_Profit_Margin,Asset_Turnover,Tax_Burder,loan_share,deposit_share,Z-score,MIACR_1day,RUONIA
18192,2016-10-01,2016,10,518,"Банк ""Первомайский"" (ПАО)",2283814.0,302594.0,2847636.0,11107.0,2232642.0,3028084.0,-77540.0,946.0,-78486.0,17141919.0,16294639.0,847280.0,5984617.0,7089798.0,8.910,8.830,8.830,138.060,142.560,44.680,-9.263290,-0.457860,0.855059,1.063368,0.049427,84.411672,16.657037,5445151.0,2023.170499,-1.441393,31.765119,101.220015,0.349122,0.435100,-0.188090,10.32,10.322381
22653,2018-10-01,2018,10,128,"ПАО ""Балтийский Банк""",3888441.0,47890.0,51103526.0,1183.0,1249533.0,55355106.0,-1612670.0,97561.0,-1710231.0,280818756.0,284034669.0,-3215913.0,15425571.0,55680040.0,0.000,0.000,0.000,569.157,738.670,0.000,53.180263,-0.609016,17.107360,1.083195,-0.011452,27.703951,19.453220,55041040.0,-8732.162717,-3.107192,19.600201,106.049657,0.054931,0.196033,-0.285736,7.27,7.260870
20443,2017-07-01,2017,7,3525,КЭБ ЭЙЧЭНБИ БАНК,269789.0,12557.0,1515323.0,0.0,174863.0,1574472.0,35777.0,6836.0,28941.0,11160815.0,10124876.0,1035939.0,1086960.0,3847919.0,37.907,36.876,36.876,78.815,120.285,0.817,2.793697,0.259309,8.733164,1.039034,0.092819,28.247996,16.227920,1797669.0,1077.362181,1.609918,16.106969,80.892752,0.097391,0.380046,0.162161,8.95,8.934762
11355,2014-04-01,2014,4,3404,"ООО ""Трастовый Республиканский Банк""",28512.0,585518.0,213832.0,729382.0,57.0,183273.0,66249.0,0.0,66249.0,3575557.0,3097243.0,478314.0,625121.0,204306.0,50.150,43.230,43.230,79.290,107.240,41.010,13.850525,1.852830,4.551919,0.857089,0.133773,305.972903,15.089632,1557244.0,747.533419,4.254247,43.552487,100.000000,0.174832,0.065964,0.914866,7.87,7.821364
1621,2011-07-01,2011,7,3062,КБ СССБ (ООО),60455.0,226549.0,171645.0,299130.0,237.0,357611.0,1122.0,800.0,322.0,3578324.0,3377678.0,200646.0,975236.0,1189334.0,0.000,0.000,0.000,116.230,112.760,95.380,0.160482,0.008999,6.174711,2.083434,0.056073,81.998497,15.090405,757779.0,1783.401613,0.042493,21.176925,28.698752,0.272540,0.352116,0.029966,3.85,3.820952


In [121]:
ans[ans.RUONIA.isna()]

,DATE,year,month,REGN,BANK,Interest_Income,Commission_Income,Operating_Income,Other_Income,Interest_Expense,Operating_Expense,EBT,Tax,Net_Profit,ASSETS,LIABILITIES,EQUITY,LOAN_PORTFOLIO,DEPOSITS,Н1.0,Н1.1,Н1.2,Н2,Н3,Н4,ROE,ROA,NIM,CIR,CAR,LTD,Bank_Size,Revenue,Leverage,Net_Profit_Margin,Asset_Turnover,Tax_Burder,loan_share,deposit_share,Z-score,MIACR_1day,RUONIA


## Key rate and Inflation

Source, the Central Bank of Russia:

https://www.cbr.ru/hd_base/infl/?UniDbQuery.Posted=True&UniDbQuery.From=17.09.2013&UniDbQuery.To=20.03.2025

In [123]:
url2 ='https://www.cbr.ru/hd_base/infl/?UniDbQuery.Posted=True&UniDbQuery.From=17.09.2013&UniDbQuery.To=20.03.2025'

RATE_INFLATION = pd.read_html(io=url2)[0]
RATE_INFLATION = RATE_INFLATION.rename(columns={'Ключевая ставка, % годовых': 'key_rate', 'Инфляция, % г/г': 'inflation', 'Дата': 'DATE'})
RATE_INFLATION['key_rate']= RATE_INFLATION['key_rate'].div(100)
RATE_INFLATION['inflation']= RATE_INFLATION['inflation'].div(100)
RATE_INFLATION = RATE_INFLATION.drop('Цель по инфляции, %', axis=1)
RATE_INFLATION['inflation_rate'] = RATE_INFLATION['inflation']* 0.01
RATE_INFLATION.head()

,DATE,key_rate,inflation,inflation_rate
0,2.2025,21.0,10.06,0.1006
1,1.2025,21.0,9.92,0.0992
2,12.2024,21.0,9.52,0.0952
3,11.2024,21.0,8.88,0.0888
4,10.2024,21.0,8.54,0.0854


In [125]:
RATE_INFLATION['year'] = RATE_INFLATION['DATE'].astype(str)
RATE_INFLATION['year'] = RATE_INFLATION['year'].apply(lambda x: x.split('.')[1])
RATE_INFLATION.loc[RATE_INFLATION['year'] == '202', 'year'] = '2020'

RATE_INFLATION['month'] = RATE_INFLATION['DATE'].astype(str)
RATE_INFLATION['month'] = RATE_INFLATION['month'].apply(lambda x: x.split('.')[0])

RATE_INFLATION

,DATE,key_rate,inflation,inflation_rate,year,month
0,2.2025,21.0,10.06,0.1006,2025,2
1,1.2025,21.0,9.92,0.0992,2025,1
2,12.2024,21.0,9.52,0.0952,2024,12
3,11.2024,21.0,8.88,0.0888,2024,11
4,10.2024,21.0,8.54,0.0854,2024,10
...,...,...,...,...,...,...
133,1.2014,5.5,6.07,0.0607,2014,1
134,12.2013,5.5,6.47,0.0647,2013,12
135,11.2013,5.5,6.50,0.0650,2013,11
136,10.2013,5.5,6.27,0.0627,2013,10


In [127]:
ans = pd.merge(ans, RATE_INFLATION[['year', 'month', 'key_rate', 'inflation']], on=['year', 'month'], how='left')
print(ans.shape)
ans.sample(5)

(31064, 44)


,DATE,year,month,REGN,BANK,Interest_Income,Commission_Income,Operating_Income,Other_Income,Interest_Expense,Operating_Expense,EBT,Tax,Net_Profit,ASSETS,LIABILITIES,EQUITY,LOAN_PORTFOLIO,DEPOSITS,Н1.0,Н1.1,Н1.2,Н2,Н3,Н4,ROE,ROA,NIM,CIR,CAR,LTD,Bank_Size,Revenue,Leverage,Net_Profit_Margin,Asset_Turnover,Tax_Burder,loan_share,deposit_share,Z-score,MIACR_1day,RUONIA,key_rate,inflation
11215,2014-04-01,2014,4,2982,ГОРБАНК,64291.0,27677.0,164657.0,128224.0,181.0,183519.0,57279.0,2771.0,54508.0,5772116.0,2737032.0,3035084.0,2547610.0,636431.0,40.030,39.300,39.300,32.920,72.160,32.150,1.795931,0.944333,2.516476,1.114553,0.525818,400.296340,15.568549,384849.0,190.179778,14.163477,6.667382,95.162276,0.441365,0.232526,0.677031,7.87,7.821364,7.50,7.33
20511,2017-10-01,2017,10,588,СУРГУТНЕФТЕГАЗБАНК,17334415.0,1567377.0,24708137.0,398116.0,15701381.0,25033012.0,1465865.0,238760.0,1227105.0,275179049.0,265539458.0,9639591.0,48878024.0,187697872.0,14.028,11.069,11.069,396.885,146.088,46.986,12.729845,0.445930,3.341039,1.013149,0.035030,26.040798,19.432933,44008045.0,2854.675567,2.788365,15.992513,83.712006,0.177623,0.706855,0.221491,8.04,8.034545,8.25,2.70
23820,2019-04-01,2019,4,2245,КОСМОС,23175.0,5123.0,25591.0,11.0,6696.0,40472.0,1950.0,489.0,1461.0,1058266.0,715839.0,342427.0,182981.0,357917.0,48.852,0.000,32.099,0.000,108.120,0.000,0.426660,0.138056,9.005853,1.581493,0.323574,51.123864,13.872142,53900.0,309.048644,2.710575,5.093237,74.923077,0.172906,0.499997,0.212589,7.75,7.673636,7.75,5.20
27016,2021-01-01,2021,1,3242,МЕЖБАНКОВСКИЙ КРЕДИТНЫЙ СОЮЗ,7280.0,21317.0,47548.0,0.0,25.0,67600.0,-12797.0,-222.0,-12575.0,778950.0,634224.0,144726.0,0.0,0.0,38.440,0.000,0.000,0.000,0.000,0.000,-8.688833,-1.614353,0.000000,1.421721,0.185796,0.000000,13.565702,76145.0,538.223954,-16.514545,9.775339,98.265218,0.000000,0.000000,-0.657875,4.31,4.284667,4.25,5.20
16230,2016-01-01,2016,1,783,РОЯЛ КРЕДИТ БАНК,423818.0,193779.0,2363339.0,2844167.0,105478.0,2774466.0,-86659.0,254.0,-102109.0,8386605.0,7771475.0,615130.0,1965804.0,2487401.0,14.350,10.180,10.180,149.310,108.000,63.300,-16.599581,-1.217525,16.193883,1.173960,0.073347,79.030442,15.942146,5825103.0,1363.387414,-1.752913,69.457224,117.828500,0.234398,0.320068,-0.526914,11.02,11.061333,11.00,9.80


In [129]:
ans[ans.inflation.isna()].year.unique()

array(['2010', '2011', '2012', '2013'], dtype=object)

In [131]:
RATE_INFLATION['quarter'] = np.select(
    [   RATE_INFLATION['month'] == '3',
        RATE_INFLATION['month'] == '6',
        RATE_INFLATION['month'] == '9',
        RATE_INFLATION['month'] == '12'
    ], ['1', '2', '3', '4'],
    default=np.nan)

RATE_INFLATION[RATE_INFLATION.month == '12']

,DATE,key_rate,inflation,inflation_rate,year,month,quarter
2,12.2024,21.00,9.52,0.0952,2024,12,4
14,12.2023,16.00,7.42,0.0742,2023,12,4
26,12.2022,7.50,11.94,0.1194,2022,12,4
38,12.2021,8.50,8.39,0.0839,2021,12,4
50,12.2020,4.25,4.90,0.0490,2020,12,4
62,12.2019,6.25,3.00,0.0300,2019,12,4
74,12.2018,7.75,4.30,0.0430,2018,12,4
86,12.2017,7.75,2.50,0.0250,2017,12,4
98,12.2016,10.00,5.40,0.0540,2016,12,4
110,12.2015,11.00,12.90,0.1290,2015,12,4


## RUB/USD

Source, the Central Bank of Russia:

https://www.cbr.ru/currency_base/dynamics/?UniDbQuery.Posted=True&UniDbQuery.so=1&UniDbQuery.mode=1&UniDbQuery.date_req1=&UniDbQuery.date_req2=&UniDbQuery.VAL_NM_RQ=R01235&UniDbQuery.From=01.01.2010&UniDbQuery.To=21.03.2025

In [133]:
url3 = 'https://www.cbr.ru/currency_base/dynamics/?UniDbQuery.Posted=True&UniDbQuery.so=1&UniDbQuery.mode=1&UniDbQuery.date_req1=&UniDbQuery.date_req2=&UniDbQuery.VAL_NM_RQ=R01235&UniDbQuery.From=01.01.2010&UniDbQuery.To=21.03.2025'

RUB_USD = pd.read_html(io=url3)[0]
RUB_USD = RUB_USD.drop(RUB_USD.loc[:2])
RUB_USD = RUB_USD.drop(1, axis=1)
RUB_USD = RUB_USD.rename(columns={0: 'DATE', 2: 'RUB_USD'}) 
RUB_USD['DATE'] = pd.to_datetime(RUB_USD['DATE'])
RUB_USD['RUB_USD'] = RUB_USD.RUB_USD.astype(int)
RUB_USD['RUB_USD']= RUB_USD['RUB_USD'].div(10000)
RUB_USD.sample(5)

C:\Users\annaz\AppData\Local\Temp\ipykernel_5092\1622755830.py:7: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  RUB_USD['DATE'] = pd.to_datetime(RUB_USD['DATE'])


,DATE,RUB_USD
60,2024-12-20,103.4207
1824,2017-10-27,57.7643
589,2022-11-03,61.6175
2471,2015-03-24,59.4452
1644,2018-07-26,63.0836


In [135]:
RUB_USD['year'] = RUB_USD['DATE'].dt.year
RUB_USD['year'] = RUB_USD['year'].astype(str)
RUB_USD['month'] = RUB_USD['DATE'].dt.month
RUB_USD['month'] = RUB_USD['month'].astype(str)

RUB_USD = RUB_USD.groupby(['year', 'month'])['RUB_USD'].mean().reset_index()
RUB_USD.sample(5)

,year,month,RUB_USD
67,2015,5,50.467958
88,2017,2,58.539361
25,2012,10,31.115650
12,2011,1,29.991920
141,2021,7,73.889891


In [137]:
ans = pd.merge(ans, RUB_USD[['year', 'month', 'RUB_USD']], on=['year', 'month'], how='left')
print(ans.shape)
ans.sample(5)

(31064, 45)


,DATE,year,month,REGN,BANK,Interest_Income,Commission_Income,Operating_Income,Other_Income,Interest_Expense,Operating_Expense,EBT,Tax,Net_Profit,ASSETS,LIABILITIES,EQUITY,LOAN_PORTFOLIO,DEPOSITS,Н1.0,Н1.1,Н1.2,Н2,Н3,Н4,ROE,ROA,NIM,CIR,CAR,LTD,Bank_Size,Revenue,Leverage,Net_Profit_Margin,Asset_Turnover,Tax_Burder,loan_share,deposit_share,Z-score,MIACR_1day,RUONIA,key_rate,inflation,RUB_USD
18205,2016-10-01,2016,10,588,СУРГУТНЕФТЕГАЗБАНК,10814558.0,1584407.0,36801666.0,57675.0,8617847.0,37983866.0,1160330.0,200199.0,960131.0,179003910.0,169914724.0,9089186.0,48254499.0,85625041.0,13.030,9.65,9.650,304.510,181.960,40.680,10.563443,0.536374,4.552344,1.032124,0.050776,56.355592,19.002918,49258306.0,1969.416293,1.949176,27.518006,82.746374,0.269572,0.503929,0.270393,10.32,10.322381,10.00,6.10,62.620005
23417,2019-01-01,2019,1,2748,БМ-БАНК,96762261.0,4754.0,104528358.0,200804.0,97890495.0,92390379.0,11009745.0,2548583.0,8461162.0,843745090.0,796705123.0,47039967.0,119896142.0,324529212.0,49.604,11.55,17.226,121.567,253.385,3.748,17.987177,1.002810,-0.941009,0.883879,0.055751,36.944638,20.553361,201496177.0,1793.677045,4.199168,23.881167,76.851571,0.142100,0.407339,0.487486,7.65,7.615294,7.75,5.00,66.512544
18447,2016-10-01,2016,10,2524,"КБ ""МКБ"" (АО)",896947.0,28440.0,3532922.0,1248.0,701698.0,3698036.0,30599.0,13942.0,16657.0,8531527.0,7449775.0,1081752.0,2371227.0,1546875.0,26.960,26.03,26.030,152.830,156.620,13.600,1.539817,0.195241,8.234091,1.046736,0.126795,153.291442,15.959279,4459557.0,788.676795,0.373512,52.271498,54.436419,0.277937,0.207640,0.148303,10.32,10.322381,10.00,6.10,62.620005
11236,2014-04-01,2014,4,3036,"""АМБ Банк"" (ПАО)",327917.0,3923194.0,4082823.0,8023994.0,165.0,3719838.0,100498.0,52293.0,48205.0,39119565.0,36626604.0,2492961.0,985404.0,9509539.0,14.320,9.59,9.590,133.530,113.150,9.040,1.933644,0.123225,33.260673,0.911095,0.063727,10.362269,17.482133,16357928.0,1569.200842,0.294689,41.815209,47.966129,0.025190,0.259635,0.086094,7.87,7.821364,7.50,7.33,35.667727
5634,2012-10-01,2012,10,992,"АО ""Вологдабанк""",65323.0,100265.0,192853.0,242186.0,225.0,268472.0,43781.0,8242.0,35539.0,1768666.0,1448374.0,320292.0,671219.0,730155.0,0.000,0.00,0.000,140.900,138.850,100.370,11.095813,2.009368,9.698474,1.392107,0.181092,91.928289,14.385736,600627.0,552.204239,5.916983,33.959323,81.174482,0.379506,0.504120,1.008746,6.10,6.022609,NaN,NaN,31.115650


In [139]:
ans[ans.RUB_USD.isna()]

,DATE,year,month,REGN,BANK,Interest_Income,Commission_Income,Operating_Income,Other_Income,Interest_Expense,Operating_Expense,EBT,Tax,Net_Profit,ASSETS,LIABILITIES,EQUITY,LOAN_PORTFOLIO,DEPOSITS,Н1.0,Н1.1,Н1.2,Н2,Н3,Н4,ROE,ROA,NIM,CIR,CAR,LTD,Bank_Size,Revenue,Leverage,Net_Profit_Margin,Asset_Turnover,Tax_Burder,loan_share,deposit_share,Z-score,MIACR_1day,RUONIA,key_rate,inflation,RUB_USD


## Reserves

Source, the Central Bank of Russia:

https://www.cbr.ru/hd_base/mrrf/mrrf_m/?UniDbQuery.Posted=True&UniDbQuery.From=01.2010&UniDbQuery.To=03.2025

Ежемесячные данные в млн.долл.США:

In [141]:
url4 = 'https://www.cbr.ru/hd_base/mrrf/mrrf_m/?UniDbQuery.Posted=True&UniDbQuery.From=01.2010&UniDbQuery.To=03.2025'

RESERVES = pd.read_html(io=url4)[0]
RESERVES.columns = RESERVES.columns.droplevel(0)
RESERVES.columns = RESERVES.columns.droplevel(0)
RESERVES = RESERVES.rename(columns={'Дата':'DATE', 'Международные резервы':'reserves', 'валютные резервы': 'forex_reserves' })
RESERVES['DATE'] = pd.to_datetime(RESERVES['DATE'])
RESERVES['reserves'] = RESERVES['reserves'].str.replace(' ', '').astype(float)
RESERVES.head(3)

,DATE,reserves,forex_reserves,иностранная валюта,счет в СДР,резервная позиция в МВФ,монетарное золото
0,2025-01-03,632355.0,414 971,387 128,23 045,4 798,217 384
1,2025-01-02,620764.0,414 239,386 377,23 057,4 805,206 526
2,2025-01-01,609068.0,413 361,385 567,23 000,4 794,195 707


In [143]:
RESERVES.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   DATE                     183 non-null    datetime64[ns]
 1   reserves                 183 non-null    float64       
 2   forex_reserves           183 non-null    object        
 3   иностранная валюта       183 non-null    object        
 4   счет в СДР               183 non-null    object        
 5   резервная позиция в МВФ  183 non-null    object        
 6   монетарное золото        183 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 10.1+ KB


In [145]:
RESERVES['year'] = RESERVES['DATE'].dt.year
RESERVES['year'] = RESERVES['year'].astype(str)
RESERVES['month'] = RESERVES['DATE'].dt.month
RESERVES['month'] = RESERVES['month'].astype(str)

RESERVES = RESERVES.groupby(['year'])['reserves'].mean().reset_index()
RESERVES.sample(5)

,year,reserves
3,2013,522836.333333
7,2017,408850.750000
6,2016,386865.416667
8,2018,455405.500000
13,2023,584884.250000


In [147]:
RESERVES.year.unique()

array(['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025'],
      dtype=object)

In [149]:
ans = pd.merge(ans, RESERVES[['year', 'reserves']], on=['year'], how='left')
print(ans.shape)
ans.sample(5)

(31064, 46)


,DATE,year,month,REGN,BANK,Interest_Income,Commission_Income,Operating_Income,Other_Income,Interest_Expense,Operating_Expense,EBT,Tax,Net_Profit,ASSETS,LIABILITIES,EQUITY,LOAN_PORTFOLIO,DEPOSITS,Н1.0,Н1.1,Н1.2,Н2,Н3,Н4,ROE,ROA,NIM,CIR,CAR,LTD,Bank_Size,Revenue,Leverage,Net_Profit_Margin,Asset_Turnover,Tax_Burder,loan_share,deposit_share,Z-score,MIACR_1day,RUONIA,key_rate,inflation,RUB_USD,reserves
8632,2013-07-01,2013,7,2617,"ООО КБ ""Банк Расчетов и Сбережений""",420479.0,237965.0,1897153.0,2117471.0,1893.0,1920658.0,236852.0,62708.0,174144.0,15721965.0,14144268.0,1577697.0,2032982.0,1125690.0,0.000,0.000,0.000,25.570,67.980,76.960,11.037861,1.107648,20.589754,1.012390,0.100350,180.598744,16.570569,4673068.0,996.513589,3.726545,29.723180,73.524395,0.129308,0.079586,0.556304,6.08,6.002174,NaN,NaN,32.740750,522836.333333
19590,2017-04-01,2017,4,2347,"ООО КБ ""ПЛАТИНА""",250867.0,390682.0,936841.0,37612.0,60943.0,1122520.0,4245.0,0.0,4245.0,6455989.0,5441706.0,1014283.0,2188782.0,472107.0,22.377,21.314,21.314,75.670,96.793,76.191,0.418522,0.065753,8.677155,1.198197,0.157107,463.619900,15.680519,1616002.0,636.507661,0.262685,25.031053,100.000000,0.339031,0.086757,0.102631,9.58,9.549500,9.75,4.10,56.435614,408850.750000
12833,2014-10-01,2014,10,3013,РАЗВИТИЕ-СТОЛИЦА,1128633.0,1700257.0,10318969.0,12861757.0,34683.0,11335479.0,32455.0,21434.0,11021.0,28950783.0,26208618.0,2742165.0,12764015.0,1642600.0,17.820,17.820,17.820,28.320,59.920,107.410,0.401909,0.038068,8.570579,1.098509,0.094718,777.061671,17.181108,26009616.0,1055.763712,0.042373,89.840803,33.957788,0.440887,0.062674,0.061150,8.24,8.230000,8.00,8.29,40.798687,470119.416667
28577,2022-01-01,2022,1,3437,Банк НФК (АО),2275154.0,268863.0,379231.0,46.0,1251301.0,1330426.0,205240.0,57133.0,148107.0,5985427.0,2773216.0,3212211.0,0.0,0.0,67.164,64.611,64.611,1959.320,1101.310,0.000,4.610749,2.474460,0.000000,3.508221,0.536672,0.000000,15.604838,2923294.0,186.333557,5.066442,48.840191,72.162834,0.000000,0.000000,1.386680,8.29,8.262500,8.50,8.73,76.594747,587225.583333
25299,2020-01-01,2020,1,3001,ПРИМОРЬЕ,8355608.0,945433.0,29958176.0,226088.0,8380225.0,29457322.0,481057.0,34707.0,446350.0,76985854.0,73904580.0,3081274.0,12609480.0,21683871.0,11.673,9.730,9.730,115.384,134.398,18.998,14.485891,0.579782,-0.195226,0.983282,0.040024,58.151425,18.159132,39485305.0,2498.507241,1.130421,51.289039,92.785262,0.163790,0.293404,0.285431,6.04,6.038824,6.25,2.40,61.812147,573888.250000


In [151]:
ans[ans.reserves.isna()]

,DATE,year,month,REGN,BANK,Interest_Income,Commission_Income,Operating_Income,Other_Income,Interest_Expense,Operating_Expense,EBT,Tax,Net_Profit,ASSETS,LIABILITIES,EQUITY,LOAN_PORTFOLIO,DEPOSITS,Н1.0,Н1.1,Н1.2,Н2,Н3,Н4,ROE,ROA,NIM,CIR,CAR,LTD,Bank_Size,Revenue,Leverage,Net_Profit_Margin,Asset_Turnover,Tax_Burder,loan_share,deposit_share,Z-score,MIACR_1day,RUONIA,key_rate,inflation,RUB_USD,reserves


## Urals
Source: https://ru.investing.com/commodities/crude-oil-urals-spot-futures-historical-data

In [153]:
URALS = pd.read_csv(r"C:\Users\annaz\Documents\Диплом\Macro data\Urals.csv")
URALS = URALS.rename(columns={'Дата': 'DATE', 'Цена': 'URALS', 'Откр.': 'open_price',
                              'Макс.': 'max_price', 'Мин.': 'min_price', 'Изм. %':'gain, %'})
URALS = URALS.drop('Объём', axis=1)
print(URALS.shape)
URALS.head()

(174, 6)


,DATE,URALS,open_price,max_price,min_price,"gain, %"
0,01.01.2025,"70,12","70,04","70,04","70,04","2,35%"
1,01.12.2024,"68,51","66,08","66,08","66,08","1,54%"
2,01.11.2024,"67,47","67,07","67,07","67,07","1,15%"
3,01.10.2024,"66,70","69,27","69,27","69,27","0,39%"
4,01.09.2024,"66,44","70,93","70,93","70,93","-10,31%"


In [155]:
URALS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   DATE        174 non-null    object
 1   URALS       174 non-null    object
 2   open_price  174 non-null    object
 3   max_price   174 non-null    object
 4   min_price   174 non-null    object
 5   gain, %     174 non-null    object
dtypes: object(6)
memory usage: 8.3+ KB


In [157]:
URALS['DATE'] = pd.to_datetime(URALS['DATE'])
URALS['year'] = URALS['DATE'].dt.year
URALS['year'] = URALS['year'].astype(str)
URALS['month'] = URALS['DATE'].dt.day
URALS['month'] = URALS['month'].astype(str)

URALS['URALS'] = URALS['URALS'].str.replace(',', '.').astype(float)
# URALS = URALS.groupby(['year', 'month'])['URALS'].mean().reset_index()
print(URALS.shape)
URALS.sample(3)

(174, 8)


,DATE,URALS,open_price,max_price,min_price,"gain, %",year,month
97,2016-01-12,53.46,"53,46","53,46","53,46","14,11%",2016,12
70,2019-01-03,68.63,"68,63","68,63","68,63","6,03%",2019,3
64,2019-01-09,59.94,"59,94","59,94","59,94","-3,46%",2019,9


In [159]:
ans = pd.merge(ans, URALS[['year', 'month', 'URALS']], on=['year', 'month'], how='left')

print(ans.shape)
ans.sample(5)

(31064, 47)


,DATE,year,month,REGN,BANK,Interest_Income,Commission_Income,Operating_Income,Other_Income,Interest_Expense,Operating_Expense,EBT,Tax,Net_Profit,ASSETS,LIABILITIES,EQUITY,LOAN_PORTFOLIO,DEPOSITS,Н1.0,Н1.1,Н1.2,Н2,Н3,Н4,ROE,ROA,NIM,CIR,CAR,LTD,Bank_Size,Revenue,Leverage,Net_Profit_Margin,Asset_Turnover,Tax_Burder,loan_share,deposit_share,Z-score,MIACR_1day,RUONIA,key_rate,inflation,RUB_USD,reserves,URALS
14671,2015-07-01,2015,7,210,"КБ ""Экспресс-кредит"" (АО)",435385.0,530653.0,4574283.0,5525800.0,338437.0,4981502.0,52248.0,1682.0,50208.0,13629548.0,12590335.0,1039213.0,3441950.0,3521428.0,10.520,7.79,7.790,36.430,100.930,88.370,4.831348,0.368376,2.816659,1.089024,0.076247,97.743018,16.427751,11066121.0,1311.525934,0.453709,81.192135,96.095544,0.252536,0.279693,0.204757,11.52,11.491739,11.50,15.6,57.179670,365182.000000,51.84
17785,2016-07-01,2016,7,2307,ИНГОССТРАХ БАНК,7702490.0,195138.0,27453501.0,169507.0,7744553.0,29104241.0,-1689454.0,24706.0,-1714160.0,157295012.0,143223493.0,14071519.0,44150232.0,45528189.0,14.920,10.20,10.200,52.510,86.720,103.180,-12.181769,-1.089774,-0.095272,1.060129,0.089459,96.973398,18.873634,35520636.0,1117.825389,-4.825814,22.582176,101.462366,0.280684,0.317882,-0.460663,10.38,10.275714,10.50,7.2,64.337968,386865.416667,39.51
26301,2020-10-01,2020,10,2,ВИКИНГ,546823.0,17419.0,1183661.0,34.0,512609.0,1209898.0,9833.0,-5516.0,15349.0,4451104.0,3808921.0,642183.0,1084308.0,450586.0,34.400,0.00,28.461,0.000,120.146,0.000,2.390129,0.344836,3.155377,1.022166,0.144275,240.643961,15.308663,1747937.0,693.120808,0.878121,39.269741,156.096817,0.243604,0.118298,0.225244,4.17,4.195000,4.25,4.0,77.663548,573888.250000,37.12
24043,2019-04-01,2019,4,3463,Ю БИ ЭС БАНК,22962.0,600248.0,3140794.0,22640.0,1.0,3021996.0,141759.0,34531.0,107228.0,25783721.0,21353960.0,4429761.0,45940.0,0.0,63.447,56.59,56.590,295055.321,3347.983,1.024,2.420627,0.415875,49.980409,0.962176,0.171805,0.000000,17.065254,3786644.0,582.056707,2.831742,14.686181,75.641053,0.001782,0.000000,0.270637,7.75,7.673636,7.75,5.2,64.603105,506857.083333,73.09
1377,2011-07-01,2011,7,2388,"АКБ ""АлтайБизнес-Банк"" (АО)",16572.0,5129.0,20741.0,22024.0,1400.0,32622.0,5126.0,178.0,4948.0,351496.0,245503.0,105993.0,205308.0,90111.0,0.000,0.00,0.000,74.980,82.860,46.980,4.668233,1.407697,7.389873,1.572827,0.301548,227.838999,12.769954,64466.0,331.621900,7.675364,18.340465,96.527507,0.584098,0.367046,0.787138,3.85,3.820952,NaN,NaN,27.912255,513469.416667,114.03


In [161]:
ans[ans.URALS.isna()]  ## отсутствуют в исходных данных

,DATE,year,month,REGN,BANK,Interest_Income,Commission_Income,Operating_Income,Other_Income,Interest_Expense,Operating_Expense,EBT,Tax,Net_Profit,ASSETS,LIABILITIES,EQUITY,LOAN_PORTFOLIO,DEPOSITS,Н1.0,Н1.1,Н1.2,Н2,Н3,Н4,ROE,ROA,NIM,CIR,CAR,LTD,Bank_Size,Revenue,Leverage,Net_Profit_Margin,Asset_Turnover,Tax_Burder,loan_share,deposit_share,Z-score,MIACR_1day,RUONIA,key_rate,inflation,RUB_USD,reserves,URALS
0,2010-07-01,2010,7,384,"НКО ""ЕРП"" (ООО)",5119.0,23151.0,5680.0,7646.0,23.0,31604.0,307.0,211.0,96.0,114968.0,104593.0,10375.0,31006.0,5048.0,0.0,0.0,0.0,0.0,0.0,0.0,0.925301,0.083501,16.435529,5.564085,0.090243,614.223455,11.652409,41596.0,1108.125301,0.230791,36.180502,31.270358,0.269692,0.048263,0.080012,2.62,2.590455,NaN,NaN,30.679113,463379.583333,NaN
1,2010-07-01,2010,7,2749,"ООО РНКО ""РИБ""",178.0,2941.0,380.0,144.0,0.0,3322.0,96.0,0.0,96.0,65463.0,57019.0,8444.0,1472.0,645.0,0.0,0.0,0.0,0.0,0.0,0.0,1.136902,0.146648,12.092391,8.742105,0.128989,228.217054,11.089240,3643.0,775.260540,2.635191,5.564976,100.000000,0.022486,0.011312,0.126936,2.62,2.590455,NaN,NaN,30.679113,463379.583333,NaN
2,2010-07-01,2010,7,2755,НК БАНК,625463.0,173983.0,3382452.0,3853313.0,24984.0,3596575.0,159807.0,23978.0,135829.0,14382586.0,12594530.0,1788056.0,6770787.0,2411766.0,0.0,0.0,0.0,0.0,0.0,0.0,7.596462,0.944399,8.868674,1.063304,0.124321,280.739798,16.481529,8035211.0,804.369997,1.690422,55.867637,84.995651,0.470763,0.191493,0.492164,2.62,2.590455,NaN,NaN,30.679113,463379.583333,NaN


## GDP
Source: https://rosstat.gov.ru/statistics/accounts

In [165]:
GDP = pd.read_excel(r"C:\Users\annaz\Documents\Диплом\VVP_kvartal_s_1995-2024.xlsx", sheet_name = 'nominal')
GDP.year = GDP.year.astype(str)
GDP.quarter = GDP.quarter.astype(str)
print(GDP.shape)
GDP.head()

(56, 3)


,year,quarter,nominal
0,2024,1,43239.971568
1,2024,2,46269.340228
2,2024,3,49591.135232
3,2024,4,NaN
4,2023,1,36176.000693


In [167]:
GDP = pd.merge(GDP, RATE_INFLATION[['year', 'quarter', 'inflation_rate']], on=['year', 'quarter'], how='left')
GDP.head()

,year,quarter,nominal,inflation_rate
0,2024,1,43239.971568,0.0772
1,2024,2,46269.340228,0.0859
2,2024,3,49591.135232,0.0863
3,2024,4,NaN,0.0952
4,2023,1,36176.000693,0.0351


In [169]:
GDP['real_GDP'] = GDP['nominal'] / (1 + GDP['inflation_rate'])
GDP.head()

,year,quarter,nominal,inflation_rate,real_GDP
0,2024,1,43239.971568,0.0772,40141.080178
1,2024,2,46269.340228,0.0859,42609.209161
2,2024,3,49591.135232,0.0863,45651.417870
3,2024,4,NaN,0.0952,NaN
4,2023,1,36176.000693,0.0351,34949.280932


In [171]:
ans['DATE'] = pd.to_datetime(ans['DATE'])
ans['shifted_date'] = ans['DATE'] - pd.DateOffset(months=3)
ans['fiscal_quarter'] = ans['shifted_date'].dt.to_period('Q')
ans['fiscal_year'] = ans['shifted_date'].dt.year
ans['custom_quarter'] = ans['fiscal_quarter'].astype(str)

ans = ans.rename(columns={'year' : 'year_initial', 'month': 'month_initial'})
ans['year'] = ans['custom_quarter'].str.extract(r'(\d{4})')
ans['quarter'] =  ans['custom_quarter'].astype(str)
ans['quarter'] = ans['quarter'].str.extract(r'Q(\d)')
ans['quarter'] = ans['quarter'].astype(str)

In [173]:
ans = pd.merge(ans, GDP[['year', 'quarter', 'real_GDP']], on=['year', 'quarter'], how='left')
ans.head()

,DATE,year_initial,month_initial,REGN,BANK,Interest_Income,Commission_Income,Operating_Income,Other_Income,Interest_Expense,Operating_Expense,EBT,Tax,Net_Profit,ASSETS,LIABILITIES,EQUITY,LOAN_PORTFOLIO,DEPOSITS,Н1.0,Н1.1,Н1.2,Н2,Н3,Н4,ROE,ROA,NIM,CIR,CAR,LTD,Bank_Size,Revenue,Leverage,Net_Profit_Margin,Asset_Turnover,Tax_Burder,loan_share,deposit_share,Z-score,MIACR_1day,RUONIA,key_rate,inflation,RUB_USD,reserves,URALS,shifted_date,fiscal_quarter,fiscal_year,custom_quarter,year,quarter,real_GDP
0,2010-07-01,2010,7,384,"НКО ""ЕРП"" (ООО)",5119.0,23151.0,5680.0,7646.0,23.0,31604.0,307.0,211.0,96.0,114968.0,104593.0,10375.0,31006.0,5048.0,0.0,0.0,0.0,0.0,0.0,0.0,0.925301,0.083501,16.435529,5.564085,0.090243,614.223455,11.652409,41596.0,1108.125301,0.230791,36.180502,31.270358,0.269692,0.048263,0.080012,2.62,2.590455,NaN,NaN,30.679113,463379.583333,NaN,2010-04-01,2010Q2,2010,2010Q2,2010,2,NaN
1,2010-07-01,2010,7,2749,"ООО РНКО ""РИБ""",178.0,2941.0,380.0,144.0,0.0,3322.0,96.0,0.0,96.0,65463.0,57019.0,8444.0,1472.0,645.0,0.0,0.0,0.0,0.0,0.0,0.0,1.136902,0.146648,12.092391,8.742105,0.128989,228.217054,11.089240,3643.0,775.260540,2.635191,5.564976,100.000000,0.022486,0.011312,0.126936,2.62,2.590455,NaN,NaN,30.679113,463379.583333,NaN,2010-04-01,2010Q2,2010,2010Q2,2010,2,NaN
2,2010-07-01,2010,7,2755,НК БАНК,625463.0,173983.0,3382452.0,3853313.0,24984.0,3596575.0,159807.0,23978.0,135829.0,14382586.0,12594530.0,1788056.0,6770787.0,2411766.0,0.0,0.0,0.0,0.0,0.0,0.0,7.596462,0.944399,8.868674,1.063304,0.124321,280.739798,16.481529,8035211.0,804.369997,1.690422,55.867637,84.995651,0.470763,0.191493,0.492164,2.62,2.590455,NaN,NaN,30.679113,463379.583333,NaN,2010-04-01,2010Q2,2010,2010Q2,2010,2,NaN
3,2010-10-01,2010,10,384,"НКО ""ЕРП"" (ООО)",6707.0,36209.0,7351.0,11732.0,32.0,47045.0,632.0,201.0,431.0,141190.0,130473.0,10717.0,26895.0,4606.0,0.0,0.0,0.0,0.0,0.0,0.0,4.021648,0.305262,24.818740,6.399810,0.075905,583.912288,11.857862,61999.0,1317.439582,0.695173,43.911750,68.196203,0.190488,0.035302,0.175534,2.69,2.646190,NaN,NaN,30.322836,463379.583333,80.92,2010-07-01,2010Q3,2010,2010Q3,2010,3,NaN
4,2010-10-01,2010,10,2179,"ОАО Банк ""ОТКРЫТИЕ""",8270906.0,25825408.0,57657914.0,79062727.0,160785.0,48862672.0,7781426.0,261767.0,7519659.0,197123028.0,178297595.0,18825433.0,54758479.0,56158479.0,0.0,0.0,0.0,0.0,0.0,0.0,39.944149,3.814703,14.810713,0.847458,0.095501,97.507055,19.099339,170816955.0,1047.110194,4.402174,86.654998,96.636002,0.277788,0.314970,1.800718,2.69,2.646190,NaN,NaN,30.322836,463379.583333,80.92,2010-07-01,2010Q3,2010,2010Q3,2010,3,NaN


In [175]:
ans[ans.real_GDP.isna()].year.unique()  ## 2010 - 2013 отсутсвуют, т.к. до сих пор нет данных по инфляции за этот период, 
# а 2024 - это только 4 квартад (данные еще не опубликованы)

array(['2010', '2011', '2012', '2013', '2024'], dtype=object)

In [177]:
ans.to_csv(r'C:\Users\annaz\Documents\Диплом\Промежуточные датафреймы\ans.csv',index = False)

## Other ideas: 

https://www.cbr.ru/hd_base/

https://www.cbr.ru/hd_base/ostat_base/?UniDbQuery.Posted=True&UniDbQuery.From=01.01.2010&UniDbQuery.To=20.03.2025

https://www.cbr.ru/hd_base/bliquidity/